<a href="https://colab.research.google.com/github/alim98/AliMikaeili.github.io/blob/master/VGG_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Essential downloads

In [1]:
!wget -O downloaded_file.zip "https://drive.usercontent.google.com/download?id=1iHPBdBOPEagvPTHZmrN__LD49emXwReY&export=download&authuser=0&confirm=t&uuid=631d60dd-569c-4bb1-a9e8-d681f0ed3d43&at=APvzH3r4me8x_LwP3n8O7lgPo8oK%3A1733988188000"



!wget -O vesicle_cloud__syn_interface__mitochondria_annotation.zip "https://drive.usercontent.google.com/download?id=1qRibZL3kr7MQJQRgDFRquHMQlIGCN4XP&export=download&authuser=0&confirm=t&uuid=631d60dd-569c-4bb1-a9e8-d681f0ed3d43&at=APvzH3r4me8x_LwP3n8O7lgPo8oK%3A1733988188000"

!unzip -q downloaded_file.zip
!unzip -q vesicle_cloud__syn_interface__mitochondria_annotation.zip

--2025-01-24 18:58:30--  https://drive.usercontent.google.com/download?id=1iHPBdBOPEagvPTHZmrN__LD49emXwReY&export=download&authuser=0&confirm=t&uuid=631d60dd-569c-4bb1-a9e8-d681f0ed3d43&at=APvzH3r4me8x_LwP3n8O7lgPo8oK%3A1733988188000
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 74.125.24.132, 2404:6800:4003:c03::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|74.125.24.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1264688649 (1.2G) [application/octet-stream]
Saving to: ‘downloaded_file.zip’

downloaded_file.zip 100%[===================>]   1.18G  51.8MB/s    in 22s     

2025-01-24 18:58:54 (54.9 MB/s) - ‘downloaded_file.zip’ saved [1264688649/1264688649]

--2025-01-24 18:58:54--  https://drive.usercontent.google.com/download?id=1qRibZL3kr7MQJQRgDFRquHMQlIGCN4XP&export=download&authuser=0&confirm=t&uuid=631d60dd-569c-4bb1-a9e8-d681f0ed3d43&at=APvzH3r4me8x_LwP3n8O7lgPo8oK%3A1733988188000
Resolving 

In [2]:

!pip install transformers scikit-learn matplotlib seaborn torch torchvision umap-learn git+https://github.com/funkelab/funlib.learn.torch.git
!pip install openpyxl


  Cloning https://github.com/funkelab/funlib.learn.torch.git to /tmp/pip-req-build-ao780g90
  Running command git clone --filter=blob:none --quiet https://github.com/funkelab/funlib.learn.torch.git /tmp/pip-req-build-ao780g90
  Resolved https://github.com/funkelab/funlib.learn.torch.git to commit 049729151c7a2c0320a446dc9d3244ac830f7ea8
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 6.6 MB/s eta 0:00:00
  Created wheel for funlib.learn.torch: filename=funlib.learn.torch-0.1.0-py3-none-any.whl size=13995 sha256=c9750311f6e3a38f37476702f22a8e521d0fe8cd0a60884da43a4904de8adc20
  Stored in directory: /tmp/pip-ephem-wheel-cache-mutattki/wheels/ae/7f/7b/ecbd355ccdfbd2bb0ab4f76ea45f60a02a572c88c1f4761e8d
Successfully built funlib.learn.torch


In [3]:

import os
import glob
import imageio.v2 as iio
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Patch, Rectangle
from torch.utils.data import Dataset, DataLoader
from transformers import ViTImageProcessor, ViTModel
import torch
import torch.nn as nn
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import seaborn as sns
from umap import UMAP
import torch.nn.functional as F

# VGG Example


#1Data

Just Uncomment either 1.1 or 1.2 for data loading

## 1.1 Dark background (sectoins that have not segmentaions be 0)

In [8]:
import os
import glob
import io
import argparse
import multiprocessing
from typing import List, Tuple

import numpy as np
import pandas as pd
import imageio.v3 as iio
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from torchvision import transforms
class SimpleVideoProcessor:
    def __init__(self, size=(80, 80), mean=(0.485,),  # Use a single channel for grayscale
                 std=(0.229,)):
        self.transform = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize(size),
            transforms.Grayscale(num_output_channels=1),  # Convert to grayscale
            transforms.ToTensor(),
            transforms.Normalize(mean=mean, std=std),
        ])

    def __call__(self, frames, return_tensors=None):
        processed_frames = [self.transform(frame) for frame in frames]
        pixel_values = torch.stack(processed_frames)
        if return_tensors == "pt":
            return {"pixel_values": pixel_values}
        else:
            return pixel_values


def load_volumes(bbox_name: str, raw_base_dir: str, seg_base_dir: str, add_mask_base_dir: str) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    raw_dir = os.path.join(raw_base_dir, bbox_name)
    seg_dir = os.path.join(seg_base_dir, bbox_name)

    if bbox_name.startswith("bbox"):
        bbox_num = bbox_name.replace("bbox", "")
        add_mask_dir = os.path.join(add_mask_base_dir, f"bbox_{bbox_num}")
    else:
        add_mask_dir = os.path.join(add_mask_base_dir, bbox_name)

    raw_tif_files = sorted(glob.glob(os.path.join(raw_dir, 'slice_*.tif')))
    seg_tif_files = sorted(glob.glob(os.path.join(seg_dir, 'slice_*.tif')))
    add_mask_tif_files = sorted(glob.glob(os.path.join(add_mask_dir, 'slice_*.tif')))

    if not (len(raw_tif_files) == len(seg_tif_files) == len(add_mask_tif_files)):
        return None, None, None

    try:
        raw_vol = np.stack([iio.imread(f) for f in raw_tif_files], axis=0)
        seg_vol = np.stack([iio.imread(f).astype(np.uint32) for f in seg_tif_files], axis=0)
        add_mask_vol = np.stack([iio.imread(f).astype(np.uint32) for f in add_mask_tif_files], axis=0)
        return raw_vol, seg_vol, add_mask_vol
    except Exception as e:
        return None, None, None

def create_segmented_cube(
    raw_vol: np.ndarray,
    seg_vol: np.ndarray,
    add_mask_vol: np.ndarray,
    central_coord: Tuple[int, int, int],
    side1_coord: Tuple[int, int, int],
    side2_coord: Tuple[int, int, int],
    segmentation_type: int,
    subvolume_size: int = 80,
    alpha: float = 0.3,
    input_mask: bool = False  # New parameter
) -> np.ndarray:
    def create_segment_masks(segmentation_volume, s1_coord, s2_coord):
        x1, y1, z1 = s1_coord
        x2, y2, z2 = s2_coord
        seg_id_1 = segmentation_volume[z1, y1, x1]
        seg_id_2 = segmentation_volume[z2, y2, x2]

        mask_1 = (segmentation_volume == seg_id_1) if seg_id_1 != 0 else np.zeros_like(segmentation_volume, dtype=bool)
        mask_2 = (segmentation_volume == seg_id_2) if seg_id_2 != 0 else np.zeros_like(segmentation_volume, dtype=bool)
        return mask_1, mask_2

    mask_1_full, mask_2_full = create_segment_masks(seg_vol, side1_coord, side2_coord)
    mask_3_full = (add_mask_vol > 0)

    half_size = subvolume_size // 2
    cx, cy, cz = central_coord
    x_start, x_end = max(cx - half_size, 0), min(cx + half_size, raw_vol.shape[2])
    y_start, y_end = max(cy - half_size, 0), min(cy + half_size, raw_vol.shape[1])
    z_start, z_end = max(cz - half_size, 0), min(cz + half_size, raw_vol.shape[0])

    sub_raw = raw_vol[z_start:z_end, y_start:y_end, x_start:x_end]
    sub_mask_1 = mask_1_full[z_start:z_end, y_start:y_end, x_start:x_end]
    sub_mask_2 = mask_2_full[z_start:z_end, y_start:y_end, x_start:x_end]
    sub_mask_3 = mask_3_full[z_start:z_end, y_start:y_end, x_start:x_end]

    pad_z = subvolume_size - sub_raw.shape[0]
    pad_y = subvolume_size - sub_raw.shape[1]
    pad_x = subvolume_size - sub_raw.shape[2]

    if pad_z > 0 or pad_y > 0 or pad_x > 0:
        sub_raw = np.pad(sub_raw, ((0, pad_z), (0, pad_y), (0, pad_x)), mode='constant', constant_values=0)
        sub_mask_1 = np.pad(sub_mask_1, ((0, pad_z), (0, pad_y), (0, pad_x)), mode='constant', constant_values=False)
        sub_mask_2 = np.pad(sub_mask_2, ((0, pad_z), (0, pad_y), (0, pad_x)), mode='constant', constant_values=False)
        sub_mask_3 = np.pad(sub_mask_3, ((0, pad_z), (0, pad_y), (0, pad_x)), mode='constant', constant_values=False)

    sub_raw = sub_raw[:subvolume_size, :subvolume_size, :subvolume_size]
    sub_mask_1 = sub_mask_1[:subvolume_size, :subvolume_size, :subvolume_size]
    sub_mask_2 = sub_mask_2[:subvolume_size, :subvolume_size, :subvolume_size]
    sub_mask_3 = sub_mask_3[:subvolume_size, :subvolume_size, :subvolume_size]

    overlaid_cube = np.zeros((subvolume_size, subvolume_size, 3, subvolume_size), dtype=np.uint8)

    for z in range(subvolume_size):
        raw_slice = sub_raw[z].astype(np.float32)
        mn, mx = raw_slice.min(), raw_slice.max()
        if mx > mn:
            raw_slice = (raw_slice - mn) / (mx - mn)
        else:
            raw_slice = raw_slice - mn

        if input_mask:  # New masking logic
            combined_mask = np.zeros_like(raw_slice, dtype=np.float32)
            if segmentation_type in [1, 3, 5]:
                combined_mask = np.logical_or(combined_mask, sub_mask_1[z])
            if segmentation_type in [2, 3, 5]:
                combined_mask = np.logical_or(combined_mask, sub_mask_2[z])
            if segmentation_type in [4, 5]:
                combined_mask = np.logical_or(combined_mask, sub_mask_3[z])

            masked_raw = raw_slice * combined_mask
            masked_rgb = np.stack([masked_raw]*3, axis=-1)
            overlaid_image = (masked_rgb * 255).astype(np.uint8)
        else:  # Original overlay logic
            raw_rgb = np.stack([raw_slice]*3, axis=-1)
            mask1_rgb = np.zeros_like(raw_rgb)
            mask2_rgb = np.zeros_like(raw_rgb)
            mask3_rgb = np.zeros_like(raw_rgb)

            if segmentation_type in [1, 3, 5]:
                mask1_rgb[sub_mask_1[z]] = [1, 0, 0]
            if segmentation_type in [2, 3, 5]:
                mask2_rgb[sub_mask_2[z]] = [0, 0, 1]
            if segmentation_type in [4, 5]:
                mask3_rgb[sub_mask_3[z]] = [0, 1, 0]

            combined_masks = mask1_rgb + mask2_rgb + mask3_rgb
            combined_masks = np.clip(combined_masks, 0, 1)
            overlaid_image = (1 - alpha) * raw_rgb + alpha * combined_masks
            overlaid_image = (np.clip(overlaid_image, 0, 1) * 255).astype(np.uint8)

        overlaid_cube[:, :, :, z] = overlaid_image

    return overlaid_cube

class VideoMAEDataset(Dataset):
    def __init__(self, vol_data_dict: dict, synapse_df: pd.DataFrame, processor,
                 segmentation_type: int, subvol_size: int = 80, num_frames: int = 16,
                 alpha: float = 0.3, input_mask: bool = False):  # Added input_mask
        self.vol_data_dict = vol_data_dict
        self.synapse_df = synapse_df.reset_index(drop=True)
        self.processor = processor
        self.segmentation_type = segmentation_type
        self.subvol_size = subvol_size
        self.num_frames = num_frames
        self.alpha = alpha
        self.input_mask = input_mask  # Store input_mask flag

    def __len__(self):
        return len(self.synapse_df)

    def __getitem__(self, idx):
        syn_info = self.synapse_df.iloc[idx]
        bbox_name = syn_info['bbox_name']
        raw_vol, seg_vol, add_mask_vol = self.vol_data_dict.get(bbox_name, (None, None, None))

        if raw_vol is None:
            return torch.zeros((self.num_frames, 3, self.subvol_size, self.subvol_size), dtype=torch.float32), syn_info, bbox_name

        central_coord = (int(syn_info['central_coord_1']), int(syn_info['central_coord_2']), int(syn_info['central_coord_3']))
        side1_coord = (int(syn_info['side_1_coord_1']), int(syn_info['side_1_coord_2']), int(syn_info['side_1_coord_3']))
        side2_coord = (int(syn_info['side_2_coord_1']), int(syn_info['side_2_coord_2']), int(syn_info['side_2_coord_3']))

        overlaid_cube = create_segmented_cube(
            raw_vol=raw_vol,
            seg_vol=seg_vol,
            add_mask_vol=add_mask_vol,
            central_coord=central_coord,
            side1_coord=side1_coord,
            side2_coord=side2_coord,
            segmentation_type=self.segmentation_type,
            subvolume_size=self.subvol_size,
            alpha=self.alpha,
            input_mask=self.input_mask  # Pass the flag
        )

        frames = [overlaid_cube[..., z] for z in range(overlaid_cube.shape[3])]
        if len(frames) < self.num_frames:
            frames += [frames[-1]] * (self.num_frames - len(frames))
        elif len(frames) > self.num_frames:
            indices = np.linspace(0, len(frames)-1, self.num_frames, dtype=int)
            frames = [frames[i] for i in indices]

        inputs = self.processor(frames, return_tensors="pt")
        return inputs["pixel_values"].squeeze(0).float(), syn_info, bbox_name

def parse_args():
    parser = argparse.ArgumentParser(description="VideoMAE Pre-training Script with Segmented Videos and Additional Masks")
    parser.add_argument('--raw_base_dir', type=str, default='raw')
    parser.add_argument('--seg_base_dir', type=str, default='seg')
    parser.add_argument('--add_mask_base_dir', type=str, default='')
    parser.add_argument('--bbox_name', type=str, default=['bbox1','bbox2','bbox3','bbox4','bbox5','bbox6', ], nargs='+')
    parser.add_argument('--excel_file', type=str, default='')
    parser.add_argument('--csv_output_dir', type=str, default='csv_outputs')
    parser.add_argument('--checkpoint_dir', type=str, default='checkpoints')
    parser.add_argument('--log_dir', type=str, default='logs')
    parser.add_argument('--size', type=tuple, default=(80,80))
    parser.add_argument('--batch_size', type=int, default=2)
    parser.add_argument('--num_epochs', type=int, default=5)
    parser.add_argument('--learning_rate', type=float, default=1e-4)
    parser.add_argument('--weight_decay', type=float, default=1e-2)
    parser.add_argument('--subvol_size', type=int, default=80)
    parser.add_argument('--num_frames', type=int, default=80)
    parser.add_argument('--mask_ratio', type=float, default=0.75)
    parser.add_argument('--patience', type=int, default=3)
    parser.add_argument('--resume_checkpoint', type=str, default=None)
    parser.add_argument('--save_gifs_dir', type=str, default='gifs')
    parser.add_argument('--num_gifs', type=int, default=10)
    parser.add_argument('--alpha', type=float, default=0.3)
    parser.add_argument('--segmentation_type', type=int, default=0, choices=range(0, 6))
    parser.add_argument('--input_mask', action='store_true', # Added argument
                       help='Mask input image using segmentation_type regions')
    args, _ = parser.parse_known_args()

    return args

def main(args):
    processor = SimpleVideoProcessor(size=(80, 80))
    vol_data_dict = {}

    for bbox_name in args.bbox_name:
        raw_vol, seg_vol, add_mask_vol = load_volumes(
            bbox_name=bbox_name,
            raw_base_dir=args.raw_base_dir,
            seg_base_dir=args.seg_base_dir,
            add_mask_base_dir=args.add_mask_base_dir
        )
        if raw_vol is not None:
            vol_data_dict[bbox_name] = (raw_vol, seg_vol, add_mask_vol)

    synapse_dfs = []
    for bbox_name in args.bbox_name:
        excel_path = os.path.join(args.excel_file, f"{bbox_name}.xlsx")
        if os.path.exists(excel_path):
            df = pd.read_excel(excel_path)
            df['bbox_name'] = bbox_name
            synapse_dfs.append(df)

    syn_df = pd.concat(synapse_dfs, ignore_index=True)
    dataset = VideoMAEDataset(
        vol_data_dict=vol_data_dict,
        synapse_df=syn_df,
        processor=processor,
        segmentation_type=args.segmentation_type,
        subvol_size=args.subvol_size,
        num_frames=args.num_frames,
        alpha=args.alpha,
        input_mask=args.input_mask  # Pass the flag
    )

    cubes = []
    syn_info_list = []
    for idx in range(len(dataset)):
        pixel_values, syn_info, _ = dataset[idx]
        cubes.append(pixel_values)
        syn_info_list.append(syn_info)

    print(f"Processed {len(cubes)} cubes successfully.")
    return cubes, pd.DataFrame(syn_info_list)

# if __name__ == "__main__":
#     args = parse_args()
#     cubes, sys_inf = main(args)
#     print(f"Final output: {len(cubes)} cubes")


# import torch
# import numpy as np
# import imageio

# cube = cubes[0]
# mean = torch.tensor([0.485, 0.456, 0.406]).view(3, 1, 1)
# std = torch.tensor([0.229, 0.224, 0.225]).view(3, 1, 1)

# denormalized_cube = cube * std + mean

# denormalized_cube = torch.clamp(denormalized_cube, 0, 1)
# frames = denormalized_cube.permute(0, 2, 3, 1).numpy()  # Change to (T, H, W, C)
# frames = (frames * 255).astype(np.uint8)  # Convert to 0-255

# imageio.mimsave('synapse_cube.gif', frames, fps=10)

# print("GIF saved successfully!")



## 1.2 Test Average (sections that have not seg. be avg pix)

In [12]:
import os
import glob
import io
import argparse
import multiprocessing
from typing import List, Tuple

import numpy as np
import pandas as pd
import imageio.v3 as iio
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from torchvision import transforms
class SimpleVideoProcessor:
    def __init__(self, size=(80, 80), mean=(0.485,),  # Use a single channel for grayscale
                 std=(0.229,)):
        self.transform = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize(size),
            transforms.Grayscale(num_output_channels=1),  # Convert to grayscale
            transforms.ToTensor(),
            transforms.Normalize(mean=mean, std=std),
        ])

    def __call__(self, frames, return_tensors=None):
        processed_frames = [self.transform(frame) for frame in frames]
        pixel_values = torch.stack(processed_frames)
        if return_tensors == "pt":
            return {"pixel_values": pixel_values}
        else:
            return pixel_values


def load_volumes(bbox_name: str, raw_base_dir: str, seg_base_dir: str, add_mask_base_dir: str) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    raw_dir = os.path.join(raw_base_dir, bbox_name)
    seg_dir = os.path.join(seg_base_dir, bbox_name)

    if bbox_name.startswith("bbox"):
        bbox_num = bbox_name.replace("bbox", "")
        add_mask_dir = os.path.join(add_mask_base_dir, f"bbox_{bbox_num}")
    else:
        add_mask_dir = os.path.join(add_mask_base_dir, bbox_name)

    raw_tif_files = sorted(glob.glob(os.path.join(raw_dir, 'slice_*.tif')))
    seg_tif_files = sorted(glob.glob(os.path.join(seg_dir, 'slice_*.tif')))
    add_mask_tif_files = sorted(glob.glob(os.path.join(add_mask_dir, 'slice_*.tif')))

    if not (len(raw_tif_files) == len(seg_tif_files) == len(add_mask_tif_files)):
        return None, None, None

    try:
        raw_vol = np.stack([iio.imread(f) for f in raw_tif_files], axis=0)
        seg_vol = np.stack([iio.imread(f).astype(np.uint32) for f in seg_tif_files], axis=0)
        add_mask_vol = np.stack([iio.imread(f).astype(np.uint32) for f in add_mask_tif_files], axis=0)
        return raw_vol, seg_vol, add_mask_vol
    except Exception as e:
        return None, None, None
def create_segmented_cube(
    raw_vol: np.ndarray,
    seg_vol: np.ndarray,
    add_mask_vol: np.ndarray,
    central_coord: Tuple[int, int, int],
    side1_coord: Tuple[int, int, int],
    side2_coord: Tuple[int, int, int],
    segmentation_type: int,
    subvolume_size: int = 80,
    alpha: float = 0.3,
    input_mask: bool = False  # New parameter
) -> np.ndarray:
    def create_segment_masks(segmentation_volume, s1_coord, s2_coord):
        x1, y1, z1 = s1_coord
        x2, y2, z2 = s2_coord
        seg_id_1 = segmentation_volume[z1, y1, x1]
        seg_id_2 = segmentation_volume[z2, y2, x2]

        mask_1 = (segmentation_volume == seg_id_1) if seg_id_1 != 0 else np.zeros_like(segmentation_volume, dtype=bool)
        mask_2 = (segmentation_volume == seg_id_2) if seg_id_2 != 0 else np.zeros_like(segmentation_volume, dtype=bool)
        return mask_1, mask_2

    mask_1_full, mask_2_full = create_segment_masks(seg_vol, side1_coord, side2_coord)
    mask_3_full = (add_mask_vol > 0)

    half_size = subvolume_size // 2
    cx, cy, cz = central_coord
    x_start, x_end = max(cx - half_size, 0), min(cx + half_size, raw_vol.shape[2])
    y_start, y_end = max(cy - half_size, 0), min(cy + half_size, raw_vol.shape[1])
    z_start, z_end = max(cz - half_size, 0), min(cz + half_size, raw_vol.shape[0])

    sub_raw = raw_vol[z_start:z_end, y_start:y_end, x_start:x_end]
    sub_mask_1 = mask_1_full[z_start:z_end, y_start:y_end, x_start:x_end]
    sub_mask_2 = mask_2_full[z_start:z_end, y_start:y_end, x_start:x_end]
    sub_mask_3 = mask_3_full[z_start:z_end, y_start:y_end, x_start:x_end]

    pad_z = subvolume_size - sub_raw.shape[0]
    pad_y = subvolume_size - sub_raw.shape[1]
    pad_x = subvolume_size - sub_raw.shape[2]

    if pad_z > 0 or pad_y > 0 or pad_x > 0:
        sub_raw = np.pad(sub_raw, ((0, pad_z), (0, pad_y), (0, pad_x)), mode='constant', constant_values=0)
        sub_mask_1 = np.pad(sub_mask_1, ((0, pad_z), (0, pad_y), (0, pad_x)), mode='constant', constant_values=False)
        sub_mask_2 = np.pad(sub_mask_2, ((0, pad_z), (0, pad_y), (0, pad_x)), mode='constant', constant_values=False)
        sub_mask_3 = np.pad(sub_mask_3, ((0, pad_z), (0, pad_y), (0, pad_x)), mode='constant', constant_values=False)

    sub_raw = sub_raw[:subvolume_size, :subvolume_size, :subvolume_size]
    sub_mask_1 = sub_mask_1[:subvolume_size, :subvolume_size, :subvolume_size]
    sub_mask_2 = sub_mask_2[:subvolume_size, :subvolume_size, :subvolume_size]
    sub_mask_3 = sub_mask_3[:subvolume_size, :subvolume_size, :subvolume_size]

    overlaid_cube = np.zeros((subvolume_size, subvolume_size, 3, subvolume_size), dtype=np.uint8)

    for z in range(subvolume_size):
        raw_slice = sub_raw[z].astype(np.float32)
        mn, mx = raw_slice.min(), raw_slice.max()
        if mx > mn:
            raw_slice = (raw_slice - mn) / (mx - mn)
        else:
            raw_slice = raw_slice - mn

        if input_mask:  # New masking logic with average pixel filling
            combined_mask = np.zeros_like(raw_slice, dtype=np.float32)
            if segmentation_type in [1, 3, 5]:
                combined_mask = np.logical_or(combined_mask, sub_mask_1[z])
            if segmentation_type in [2, 3, 5]:
                combined_mask = np.logical_or(combined_mask, sub_mask_2[z])
            if segmentation_type in [4, 5]:
                combined_mask = np.logical_or(combined_mask, sub_mask_3[z])

            # Fill the non-segmented regions with the average of the segmented pixels
            if np.any(~combined_mask):  # Non-masked areas
                avg_value = np.mean(raw_slice[combined_mask == 0])  # Average of segmented pixels
                raw_slice[combined_mask == 0] = avg_value  # Replace non-segmented areas with the average

            # Convert to RGB (average pixel value in all channels)
            masked_rgb = np.stack([raw_slice] * 3, axis=-1)
            overlaid_image = (masked_rgb * 255).astype(np.uint8)
        else:  # Original overlay logic
            raw_rgb = np.stack([raw_slice] * 3, axis=-1)
            mask1_rgb = np.zeros_like(raw_rgb)
            mask2_rgb = np.zeros_like(raw_rgb)
            mask3_rgb = np.zeros_like(raw_rgb)

            if segmentation_type in [1, 3, 5]:
                mask1_rgb[sub_mask_1[z]] = [1, 0, 0]
            if segmentation_type in [2, 3, 5]:
                mask2_rgb[sub_mask_2[z]] = [0, 0, 1]
            if segmentation_type in [4, 5]:
                mask3_rgb[sub_mask_3[z]] = [0, 1, 0]

            combined_masks = mask1_rgb + mask2_rgb + mask3_rgb
            combined_masks = np.clip(combined_masks, 0, 1)
            overlaid_image = (1 - alpha) * raw_rgb + alpha * combined_masks
            overlaid_image = (np.clip(overlaid_image, 0, 1) * 255).astype(np.uint8)

        overlaid_cube[:, :, :, z] = overlaid_image

    return overlaid_cube


class VideoMAEDataset(Dataset):
    def __init__(self, vol_data_dict: dict, synapse_df: pd.DataFrame, processor,
                 segmentation_type: int, subvol_size: int = 80, num_frames: int = 16,
                 alpha: float = 0.3, input_mask: bool = False):  # Added input_mask
        self.vol_data_dict = vol_data_dict
        self.synapse_df = synapse_df.reset_index(drop=True)
        self.processor = processor
        self.segmentation_type = segmentation_type
        self.subvol_size = subvol_size
        self.num_frames = num_frames
        self.alpha = alpha
        self.input_mask = input_mask  # Store input_mask flag

    def __len__(self):
        return len(self.synapse_df)

    def __getitem__(self, idx):
        syn_info = self.synapse_df.iloc[idx]
        bbox_name = syn_info['bbox_name']
        raw_vol, seg_vol, add_mask_vol = self.vol_data_dict.get(bbox_name, (None, None, None))

        if raw_vol is None:
            return torch.zeros((self.num_frames, 3, self.subvol_size, self.subvol_size), dtype=torch.float32), syn_info, bbox_name

        central_coord = (int(syn_info['central_coord_1']), int(syn_info['central_coord_2']), int(syn_info['central_coord_3']))
        side1_coord = (int(syn_info['side_1_coord_1']), int(syn_info['side_1_coord_2']), int(syn_info['side_1_coord_3']))
        side2_coord = (int(syn_info['side_2_coord_1']), int(syn_info['side_2_coord_2']), int(syn_info['side_2_coord_3']))

        overlaid_cube = create_segmented_cube(
            raw_vol=raw_vol,
            seg_vol=seg_vol,
            add_mask_vol=add_mask_vol,
            central_coord=central_coord,
            side1_coord=side1_coord,
            side2_coord=side2_coord,
            segmentation_type=self.segmentation_type,
            subvolume_size=self.subvol_size,
            alpha=self.alpha,
            input_mask=self.input_mask  # Pass the flag
        )

        frames = [overlaid_cube[..., z] for z in range(overlaid_cube.shape[3])]
        if len(frames) < self.num_frames:
            frames += [frames[-1]] * (self.num_frames - len(frames))
        elif len(frames) > self.num_frames:
            indices = np.linspace(0, len(frames)-1, self.num_frames, dtype=int)
            frames = [frames[i] for i in indices]

        inputs = self.processor(frames, return_tensors="pt")
        return inputs["pixel_values"].squeeze(0).float(), syn_info, bbox_name

def parse_args():
    parser = argparse.ArgumentParser(description="VideoMAE Pre-training Script with Segmented Videos and Additional Masks")
    parser.add_argument('--raw_base_dir', type=str, default='raw')
    parser.add_argument('--seg_base_dir', type=str, default='seg')
    parser.add_argument('--add_mask_base_dir', type=str, default='')
    parser.add_argument('--bbox_name', type=str, default=['bbox1','bbox2','bbox3','bbox4','bbox5','bbox6', ], nargs='+')
    parser.add_argument('--excel_file', type=str, default='')
    parser.add_argument('--csv_output_dir', type=str, default='csv_outputs')
    parser.add_argument('--checkpoint_dir', type=str, default='checkpoints')
    parser.add_argument('--log_dir', type=str, default='logs')
    parser.add_argument('--size', type=tuple, default=(80,80))
    parser.add_argument('--batch_size', type=int, default=2)
    parser.add_argument('--num_epochs', type=int, default=5)
    parser.add_argument('--learning_rate', type=float, default=1e-4)
    parser.add_argument('--weight_decay', type=float, default=1e-2)
    parser.add_argument('--subvol_size', type=int, default=80)
    parser.add_argument('--num_frames', type=int, default=80)
    parser.add_argument('--mask_ratio', type=float, default=0.75)
    parser.add_argument('--patience', type=int, default=3)
    parser.add_argument('--resume_checkpoint', type=str, default=None)
    parser.add_argument('--save_gifs_dir', type=str, default='gifs')
    parser.add_argument('--num_gifs', type=int, default=10)
    parser.add_argument('--alpha', type=float, default=0.3)
    parser.add_argument('--segmentation_type', type=int, default=5, choices=range(0, 6))
    parser.add_argument('--input_mask', action='store_true', # Added argument
                       help='Mask input image using segmentation_type regions')
    args, _ = parser.parse_known_args()

    return args

def main(args):
    processor = SimpleVideoProcessor(size=(80, 80))
    vol_data_dict = {}

    for bbox_name in args.bbox_name:
        raw_vol, seg_vol, add_mask_vol = load_volumes(
            bbox_name=bbox_name,
            raw_base_dir=args.raw_base_dir,
            seg_base_dir=args.seg_base_dir,
            add_mask_base_dir=args.add_mask_base_dir
        )
        if raw_vol is not None:
            vol_data_dict[bbox_name] = (raw_vol, seg_vol, add_mask_vol)

    synapse_dfs = []
    for bbox_name in args.bbox_name:
        excel_path = os.path.join(args.excel_file, f"{bbox_name}.xlsx")
        if os.path.exists(excel_path):
            df = pd.read_excel(excel_path)
            df['bbox_name'] = bbox_name
            synapse_dfs.append(df)

    syn_df = pd.concat(synapse_dfs, ignore_index=True)
    dataset = VideoMAEDataset(
        vol_data_dict=vol_data_dict,
        synapse_df=syn_df,
        processor=processor,
        segmentation_type=args.segmentation_type,
        subvol_size=args.subvol_size,
        num_frames=args.num_frames,
        alpha=args.alpha,
        input_mask=args.input_mask  # Pass the flag
    )

    cubes = []
    syn_info_list = []
    for idx in range(len(dataset)):
        pixel_values, syn_info, _ = dataset[idx]
        cubes.append(pixel_values)
        syn_info_list.append(syn_info)

    print(f"Processed {len(cubes)} cubes successfully.")
    return cubes, pd.DataFrame(syn_info_list)

# if __name__ == "__main__":
#     args = parse_args()
#     args.input_mask=True
#     cubes, sys_inf = main(args)
#     print(f"Final output: {len(cubes)} cubes")

# # Visualizeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee
# import torch
# import numpy as np
# import imageio

# cube = cubes[0]
# mean = torch.tensor([0.485, 0.456, 0.406]).view(3, 1, 1)
# std = torch.tensor([0.229, 0.224, 0.225]).view(3, 1, 1)

# denormalized_cube = cube * std + mean

# denormalized_cube = torch.clamp(denormalized_cube, 0, 1)
# frames = denormalized_cube.permute(0, 2, 3, 1).numpy()  # Change to (T, H, W, C)
# frames = (frames * 255).astype(np.uint8)  # Convert to 0-255

# imageio.mimsave('synapse_cube.gif', frames, fps=10)

# print("GIF saved successfully!")



# VGG Model and Feature Extracture

In [5]:

import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader
import pandas as pd
import os

class Vgg3D(nn.Module):
    def __init__(
        self,
        input_size=(80, 80, 80),
        fmaps=24,
        downsample_factors=[(2, 2, 2), (2, 2, 2), (2, 2, 2), (2, 2, 2)],
        fmap_inc=(2, 2, 2, 2),
        n_convolutions=(4, 2, 2, 2),
        output_classes=7,
        input_fmaps=3,
    ):
        super(Vgg3D, self).__init__()

        # Validate input parameters
        if len(downsample_factors) != len(fmap_inc):
            raise ValueError("fmap_inc needs to have same length as downsample factors")
        if len(n_convolutions) != len(fmap_inc):
            raise ValueError("n_convolutions needs to have the same length as downsample factors")
        if np.any(np.array(n_convolutions) < 1):
            raise ValueError("Each layer must have at least one convolution")

        current_fmaps = input_fmaps
        current_size = np.array(input_size)

        # Feature extraction layers
        layers = []
        for i, (df, nc) in enumerate(zip(downsample_factors, n_convolutions)):
            # Convolution block
            layers += [
                nn.Conv3d(current_fmaps, fmaps, kernel_size=3, padding=1),
                nn.BatchNorm3d(fmaps),
                nn.ReLU(inplace=True)
            ]

            # Additional convolutions
            for _ in range(nc - 1):
                layers += [
                    nn.Conv3d(fmaps, fmaps, kernel_size=3, padding=1),
                    nn.BatchNorm3d(fmaps),
                    nn.ReLU(inplace=True)
                ]

            # Downsampling
            layers.append(nn.MaxPool3d(df))

            # Update feature map size
            current_fmaps = fmaps
            fmaps *= fmap_inc[i]

            # Update spatial dimensions
            current_size = np.floor(current_size / np.array(df))
            # logger.info(f"Block {i+1}: features {current_fmaps}, size {current_size}")

        self.features = nn.Sequential(*layers)

        # Classifier (not used for feature extraction)
        self.classifier = nn.Sequential(
            nn.Linear(int(np.prod(current_size)) * current_fmaps, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, output_classes),
        )

    def forward(self, x, return_features=False):
        x = self.features(x)
        if return_features:
            return x  # Return raw features before flattening
        x = x.view(x.size(0), -1)
        return self.classifier(x)


def extract_features(model, dataset, args):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device).eval()

    dataloader = DataLoader(
        dataset,
        batch_size=args.batch_size,
        num_workers=2,
        collate_fn=lambda b: (
            torch.stack([item[0] for item in b]),  # Pixel values
            [item[1] for item in b],               # Synapse info
            [item[2] for item in b]                # Bbox names
        )
    )

    features = []
    metadata = []

    with torch.no_grad():
        for batch in dataloader:
            pixels, info, names = batch
            inputs = pixels.permute(0, 2, 1, 3, 4).to(device)  # Reshape for 3D convolution

            batch_features = model.features(inputs)
            pooled_features = nn.AdaptiveAvgPool3d((1, 1, 1))(batch_features)

            # Flatten and convert to numpy immediately
            features.append(pooled_features.cpu().numpy().squeeze())
            metadata.extend(zip(names, info))

    # Combine all batch features
    features = np.concatenate(features, axis=0)

    # Create metadata DataFrame
    metadata_df = pd.DataFrame([
        {"bbox": name, **info.to_dict()}
        for name, info in metadata
    ])

    # Create feature columns
    feature_columns = [f'feat_{i+1}' for i in range(features.shape[1])]
    features_df = pd.DataFrame(features, columns=feature_columns)

    # Combine with metadata
    combined_df = pd.concat([metadata_df, features_df], axis=1)

    return combined_df
# # Initialize the model
# model = Vgg3D(input_size=(80, 80, 80), fmaps=24, downsample_factors=[(2, 2, 2), (2, 2, 2)],
#               fmap_inc=(2, 2), n_convolutions=(4, 2), output_classes=7, input_fmaps=1)

# Load model from checkpoint
def load_model_from_checkpoint(model, checkpoint_path):
    # Load the checkpoint
    checkpoint = torch.load(checkpoint_path)

    # Load the state_dict into the model
    model.load_state_dict(checkpoint['model_state_dict'])

    # If the checkpoint includes the optimizer state, you can load that as well
    # model.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])  # if needed

    # Set the model to evaluation mode (disable dropout, batch norm updates, etc.)
    model.eval()

    return model


# Example: Load your model with the checkpoint
model = Vgg3D(input_size=(80, 80, 80), fmaps=24,output_classes=7, input_fmaps=1)

# Paths and directories
checkpoint_url = "https://dl.dropboxusercontent.com/scl/fo/mfejaomhu43aa6oqs6zsf/AKMAAgT7OrUtruR0AQXZBy0/hemibrain_production.checkpoint.20220225?rlkey=6cmwxdvehy4ylztvsbgkfnrfc&dl=0"
checkpoint_path = 'hemibrain_production.checkpoint'

# Download the checkpoint if it doesn't exist
if not os.path.exists(checkpoint_path):
    os.system(f"wget -O {checkpoint_path} '{checkpoint_url}'")
    print("Downloaded VGG3D checkpoint.")
else:
    print("VGG3D checkpoint already exists.")


checkpoint_path = 'hemibrain_production.checkpoint'  # Replace with the actual path to your checkpoint
model = load_model_from_checkpoint(model, checkpoint_path)


Downloaded VGG3D checkpoint.


<ipython-input-5-16434c063c55>:136: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path)


In [6]:
!mkdir Result


#raw

In [9]:

args = parse_args()
args.input_mask=False
args.segmentation_type=0
# Initialize the arguments (or use your existing args)
# Use the main function to load data and prepare the dataset
# cubes, sys_inf = main(args)

# Define the processor (if not already done)
processor = SimpleVideoProcessor(size=(80, 80))

# Prepare the dataset
vol_data_dict = {}
for bbox_name in args.bbox_name:
    raw_vol, seg_vol, add_mask_vol = load_volumes(
        bbox_name=bbox_name,
        raw_base_dir=args.raw_base_dir,
        seg_base_dir=args.seg_base_dir,
        add_mask_base_dir=args.add_mask_base_dir
    )
    if raw_vol is not None:
        vol_data_dict[bbox_name] = (raw_vol, seg_vol, add_mask_vol)

synapse_dfs = []
for bbox_name in args.bbox_name:
    excel_path = os.path.join(args.excel_file, f"{bbox_name}.xlsx")
    if os.path.exists(excel_path):
        df = pd.read_excel(excel_path)
        df['bbox_name'] = bbox_name
        synapse_dfs.append(df)

syn_df = pd.concat(synapse_dfs, ignore_index=True)

# Prepare the dataset
dataset = VideoMAEDataset(
    vol_data_dict=vol_data_dict,
    synapse_df=syn_df,
    processor=processor,
    segmentation_type=args.segmentation_type,
    subvol_size=args.subvol_size,
    num_frames=args.num_frames,
    alpha=args.alpha,
    input_mask=False
)

# Now extract features from the model
features_df = extract_features(model, dataset, args)

# Print the resulting DataFrame
# print(features_df.head())
import os
import torch
import numpy as np
import imageio

# Extract the data tensor (cube) from the tuple returned by dataset[0]
cube, syn_info, bbox_name = dataset[0]

# Define the mean and std values for denormalization
mean = torch.tensor([0.485, 0.456, 0.406]).view(1, 3, 1, 1)  # 1x3x1x1 for RGB
std = torch.tensor([0.229, 0.224, 0.225]).view(1, 3, 1, 1)  # 1x3x1x1 for RGB

# Denormalize the cube (inverting the normalization step)
denormalized_cube = cube * std + mean

# Clamp the values between 0 and 1
denormalized_cube = torch.clamp(denormalized_cube, 0, 1)

# Convert to numpy and adjust the shape (T, H, W, C) for gif creation
frames = denormalized_cube.permute(0, 2, 3, 1).numpy()  # Change to (T, H, W, C)

# Convert frames to 0-255 range for gif
frames = (frames * 255).astype(np.uint8)

Gif_Name = f"VGG_Gif_segmentation_type_{args.segmentation_type}_input_mask_{args.input_mask}"
# Gif_Name = f"VGG_Gif_segmentation_type_{args.segmentation_type}_input_mask_True"
output_gif_path = os.path.join("Result", f"{Gif_Name}.gif")
# Save the frames as a gif using imageio
imageio.mimsave(output_gif_path, frames, fps=10)

print("GIF saved successfully!")

segmentation_type_name = f"VGG_CSV_segmentation_type_{args.segmentation_type}_input_mask_{args.input_mask}"
# segmentation_type_name = f"VGG_CSV_segmentation_type_{args.segmentation_type}_input_mask_True"
output_csv_path = os.path.join("Result", f"{segmentation_type_name}_features.csv")

# Save the features_df DataFrame as a CSV file
features_df.to_csv(output_csv_path, index=False)

print(f"Features saved successfully as {output_csv_path}")


GIF saved successfully!
Features saved successfully as Result/VGG_CSV_segmentation_typeavg_0_input_mask_False_features.csv


# Seg 5 False

In [10]:

# checkpoint_path = 'hemibrain_production.checkpoint'  # Replace with the actual path to your checkpoint
# model = load_model_from_checkpoint(model, checkpoint_path)

# Initialize the arguments (or use your existing args)
args = parse_args()

args.input_mask=False
args.segmentation_type=5
# Use the main function to load data and prepare the dataset
cubes, sys_inf = main(args)

# Define the processor (if not already done)
processor = SimpleVideoProcessor(size=(80, 80))

# Prepare the dataset
vol_data_dict = {}
for bbox_name in args.bbox_name:
    raw_vol, seg_vol, add_mask_vol = load_volumes(
        bbox_name=bbox_name,
        raw_base_dir=args.raw_base_dir,
        seg_base_dir=args.seg_base_dir,
        add_mask_base_dir=args.add_mask_base_dir
    )
    if raw_vol is not None:
        vol_data_dict[bbox_name] = (raw_vol, seg_vol, add_mask_vol)

synapse_dfs = []
for bbox_name in args.bbox_name:
    excel_path = os.path.join(args.excel_file, f"{bbox_name}.xlsx")
    if os.path.exists(excel_path):
        df = pd.read_excel(excel_path)
        df['bbox_name'] = bbox_name
        synapse_dfs.append(df)

syn_df = pd.concat(synapse_dfs, ignore_index=True)

# Prepare the dataset
dataset = VideoMAEDataset(
    vol_data_dict=vol_data_dict,
    synapse_df=syn_df,
    processor=processor,
    segmentation_type=5,
    subvol_size=args.subvol_size,
    num_frames=args.num_frames,
    alpha=args.alpha,
    input_mask=False
)

# Now extract features from the model
features_df = extract_features(model, dataset, args)

# Print the resulting DataFrame
# print(features_df.head())
import os
import torch
import numpy as np
import imageio

# Extract the data tensor (cube) from the tuple returned by dataset[0]
cube, syn_info, bbox_name = dataset[0]

# Define the mean and std values for denormalization
mean = torch.tensor([0.485, 0.456, 0.406]).view(1, 3, 1, 1)  # 1x3x1x1 for RGB
std = torch.tensor([0.229, 0.224, 0.225]).view(1, 3, 1, 1)  # 1x3x1x1 for RGB

# Denormalize the cube (inverting the normalization step)
denormalized_cube = cube * std + mean

# Clamp the values between 0 and 1
denormalized_cube = torch.clamp(denormalized_cube, 0, 1)

# Convert to numpy and adjust the shape (T, H, W, C) for gif creation
frames = denormalized_cube.permute(0, 2, 3, 1).numpy()  # Change to (T, H, W, C)

# Convert frames to 0-255 range for gif
frames = (frames * 255).astype(np.uint8)

Gif_Name = f"VGG_Gif_segmentation_type_{args.segmentation_type}_input_mask_{args.input_mask}"
# Gif_Name = "VGG_Gif_segmentation_type_5_input_mask_False"
output_gif_path = os.path.join("Result", f"{Gif_Name}.gif")
# Save the frames as a gif using imageio
imageio.mimsave(output_gif_path, frames, fps=10)

print("GIF saved successfully!")

segmentation_type_name = f"VGG_CSV_segmentation_type_{args.segmentation_type}_input_mask_{args.input_mask}"
# segmentation_type_name = "VGG_CSV_segmentation_type_5_input_mask_False"
output_csv_path = os.path.join("Result", f"{segmentation_type_name}_features.csv")

# Save the features_df DataFrame as a CSV file
features_df.to_csv(output_csv_path, index=False)

print(f"Features saved successfully as {output_csv_path}")


Processed 444 cubes successfully.
GIF saved successfully!
Features saved successfully as Result/VGG_CSV_segmentation_type_5_input_mask_False_features.csv


# Seg5 True Black

In [11]:
args = parse_args()

args.input_mask=True
args.segmentation_type=5
# Initialize the arguments (or use your existing args)

# Use the main function to load data and prepare the dataset
# cubes, sys_inf = main(args)

# Define the processor (if not already done)
processor = SimpleVideoProcessor(size=(80, 80))

# Prepare the dataset
vol_data_dict = {}
for bbox_name in args.bbox_name:
    raw_vol, seg_vol, add_mask_vol = load_volumes(
        bbox_name=bbox_name,
        raw_base_dir=args.raw_base_dir,
        seg_base_dir=args.seg_base_dir,
        add_mask_base_dir=args.add_mask_base_dir
    )
    if raw_vol is not None:
        vol_data_dict[bbox_name] = (raw_vol, seg_vol, add_mask_vol)

synapse_dfs = []
for bbox_name in args.bbox_name:
    excel_path = os.path.join(args.excel_file, f"{bbox_name}.xlsx")
    if os.path.exists(excel_path):
        df = pd.read_excel(excel_path)
        df['bbox_name'] = bbox_name
        synapse_dfs.append(df)

syn_df = pd.concat(synapse_dfs, ignore_index=True)

# Prepare the dataset
dataset = VideoMAEDataset(
    vol_data_dict=vol_data_dict,
    synapse_df=syn_df,
    processor=processor,
    segmentation_type=5,
    subvol_size=args.subvol_size,
    num_frames=args.num_frames,
    alpha=args.alpha,
    input_mask=True
)

# Now extract features from the model
features_df = extract_features(model, dataset, args)

# Print the resulting DataFrame
# print(features_df.head())
import os
import torch
import numpy as np
import imageio

# Extract the data tensor (cube) from the tuple returned by dataset[0]
cube, syn_info, bbox_name = dataset[0]

# Define the mean and std values for denormalization
mean = torch.tensor([0.485, 0.456, 0.406]).view(1, 3, 1, 1)  # 1x3x1x1 for RGB
std = torch.tensor([0.229, 0.224, 0.225]).view(1, 3, 1, 1)  # 1x3x1x1 for RGB

# Denormalize the cube (inverting the normalization step)
denormalized_cube = cube * std + mean

# Clamp the values between 0 and 1
denormalized_cube = torch.clamp(denormalized_cube, 0, 1)

# Convert to numpy and adjust the shape (T, H, W, C) for gif creation
frames = denormalized_cube.permute(0, 2, 3, 1).numpy()  # Change to (T, H, W, C)

# Convert frames to 0-255 range for gif
frames = (frames * 255).astype(np.uint8)

Gif_Name = f"VGG_Gif_segmentation_type_{args.segmentation_type}_input_mask_{args.input_mask}"
# Gif_Name = "VGG_Gif_segmentation_type_5_input_mask_True"
output_gif_path = os.path.join("Result", f"{Gif_Name}.gif")
# Save the frames as a gif using imageio
imageio.mimsave(output_gif_path, frames, fps=10)

print("GIF saved successfully!")

segmentation_type_name = f"VGG_CSV_segmentation_type_{args.segmentation_type}_input_mask_{args.input_mask}_Black"
# segmentation_type_name = "VGG_CSV_segmentation_type_5_input_mask_True"
output_csv_path = os.path.join("Result", f"{segmentation_type_name}_Black_features.csv")

# Save the features_df DataFrame as a CSV file
features_df.to_csv(output_csv_path, index=False)

print(f"Features saved successfully as {output_csv_path}")


Processed 444 cubes successfully.
GIF saved successfully!
Features saved successfully as Result/VGG_CSV_segmentation_type_5_input_mask_True_Black_Black_features.csv


# seg5 True Avg

## Before run this run 1.2

In [13]:

# checkpoint_path = 'hemibrain_production.checkpoint'  # Replace with the actual path to your checkpoint
# model = load_model_from_checkpoint(model, checkpoint_path)

# Initialize the arguments (or use your existing args)
args = parse_args()

args.input_mask=True
args.segmentation_type=5
# Use the main function to load data and prepare the dataset
# cubes, sys_inf = main(args)

# Define the processor (if not already done)
processor = SimpleVideoProcessor(size=(80, 80))

# Prepare the dataset
vol_data_dict = {}
for bbox_name in args.bbox_name:
    raw_vol, seg_vol, add_mask_vol = load_volumes(
        bbox_name=bbox_name,
        raw_base_dir=args.raw_base_dir,
        seg_base_dir=args.seg_base_dir,
        add_mask_base_dir=args.add_mask_base_dir
    )
    if raw_vol is not None:
        vol_data_dict[bbox_name] = (raw_vol, seg_vol, add_mask_vol)

synapse_dfs = []
for bbox_name in args.bbox_name:
    excel_path = os.path.join(args.excel_file, f"{bbox_name}.xlsx")
    if os.path.exists(excel_path):
        df = pd.read_excel(excel_path)
        df['bbox_name'] = bbox_name
        synapse_dfs.append(df)

syn_df = pd.concat(synapse_dfs, ignore_index=True)

# Prepare the dataset
dataset = VideoMAEDataset(
    vol_data_dict=vol_data_dict,
    synapse_df=syn_df,
    processor=processor,
    segmentation_type=5,
    subvol_size=args.subvol_size,
    num_frames=args.num_frames,
    alpha=args.alpha,
    input_mask=True
)

# Now extract features from the model
features_df = extract_features(model, dataset, args)

# Print the resulting DataFrame
# print(features_df.head())
import os
import torch
import numpy as np
import imageio

# Extract the data tensor (cube) from the tuple returned by dataset[0]
cube, syn_info, bbox_name = dataset[0]

# Define the mean and std values for denormalization
mean = torch.tensor([0.485, 0.456, 0.406]).view(1, 3, 1, 1)  # 1x3x1x1 for RGB
std = torch.tensor([0.229, 0.224, 0.225]).view(1, 3, 1, 1)  # 1x3x1x1 for RGB

# Denormalize the cube (inverting the normalization step)
denormalized_cube = cube * std + mean

# Clamp the values between 0 and 1
denormalized_cube = torch.clamp(denormalized_cube, 0, 1)

# Convert to numpy and adjust the shape (T, H, W, C) for gif creation
frames = denormalized_cube.permute(0, 2, 3, 1).numpy()  # Change to (T, H, W, C)

# Convert frames to 0-255 range for gif
frames = (frames * 255).astype(np.uint8)

# Gif_Name = f"VGG_Gif_segmentation_type_{args.segmentation_type}_input_mask_{args.input_mask}"
Gif_Name = "VGG_Gif_segmentation_type_5_input_mask_True_AVG"
output_gif_path = os.path.join("Result", f"{Gif_Name}.gif")
# Save the frames as a gif using imageio
imageio.mimsave(output_gif_path, frames, fps=10)

print("GIF saved successfully!")

# segmentation_type_name = f"VGG_CSV_segmentation_type_{args.segmentation_type}_input_mask_{args.input_mask}"
segmentation_type_name = "VGG_CSV_segmentation_type_5_input_mask_True_AVG"
output_csv_path = os.path.join("Result", f"{segmentation_type_name}_features.csv")

# Save the features_df DataFrame as a CSV file
features_df.to_csv(output_csv_path, index=False)

print(f"Features saved successfully as {output_csv_path}")


GIF saved successfully!
Features saved successfully as Result/VGG_CSV_segmentation_type_5_input_mask_True_AVG_features.csv


In [17]:
import shutil

# Replace 'folder_name' with the name of your folder
shutil.make_archive('/content/Result', 'zip', '/content/Result')
from google.colab import files

# Download the zip file
files.download('/content/Result.zip')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [15]:
!dir

bbox_1	    bbox_7
bbox1.xlsx  bbox7.xlsx
bbox_2	    downloaded_file.zip
bbox2.xlsx  hemibrain_production.checkpoint
bbox_3	    __MACOSX
bbox3.xlsx  raw
bbox_4	    Result
bbox4.xlsx  Result.zip
bbox_5	    sample_data
bbox5.xlsx  seg
bbox_6	    vesicle_cloud__syn_interface__mitochondria_annotation.zip
bbox6.xlsx


#Fuck that

In [ ]:
# import os
# import glob
# import io
# import argparse
# import multiprocessing
# from typing import List, Tuple

# import numpy as np
# import pandas as pd
# import imageio.v3 as iio
# import torch
# from torch.utils.data import Dataset, DataLoader
# from sklearn.model_selection import train_test_split
# from torchvision import transforms

# class SimpleVideoProcessor:
#     def __init__(self, size=(80, 80),
#                  mean=(0.485,),  # Single channel default
#                  std=(0.229,),
#                  grayscale=True):
#         self.grayscale = grayscale
#         transform_chain = [
#             transforms.ToPILImage(),
#             transforms.Resize(size),
#         ]

#         if grayscale:
#             transform_chain.append(transforms.Grayscale(num_output_channels=1))

#         transform_chain += [
#             transforms.ToTensor(),
#             transforms.Normalize(mean=mean, std=std),
#         ]

#         self.transform = transforms.Compose(transform_chain)

#     def __call__(self, frames, return_tensors=None):
#         processed_frames = [self.transform(frame) for frame in frames]
#         pixel_values = torch.stack(processed_frames)
#         return {"pixel_values": pixel_values} if return_tensors == "pt" else pixel_values


# def load_volumes(bbox_name: str, raw_base_dir: str, seg_base_dir: str, add_mask_base_dir: str) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
#     raw_dir = os.path.join(raw_base_dir, bbox_name)
#     seg_dir = os.path.join(seg_base_dir, bbox_name)
#     # print(raw_dir)
#     if bbox_name.startswith("bbox"):
#         bbox_num = bbox_name.replace("bbox", "")
#         add_mask_dir = os.path.join(add_mask_base_dir, f"bbox_{bbox_num}")
#     else:
#         add_mask_dir = os.path.join(add_mask_base_dir, bbox_name)

#     raw_tif_files = sorted(glob.glob(os.path.join(raw_dir, 'slice_*.tif')))
#     seg_tif_files = sorted(glob.glob(os.path.join(seg_dir, 'slice_*.tif')))
#     add_mask_tif_files = sorted(glob.glob(os.path.join(add_mask_dir, 'slice_*.tif')))

#     if not (len(raw_tif_files) == len(seg_tif_files) == len(add_mask_tif_files)):
#         print(len(raw_tif_files) , len(seg_tif_files) , len(add_mask_tif_files))
#         return None, None, None

#     try:
#         raw_vol = np.stack([iio.imread(f) for f in raw_tif_files], axis=0)
#         seg_vol = np.stack([iio.imread(f).astype(np.uint32) for f in seg_tif_files], axis=0)
#         add_mask_vol = np.stack([iio.imread(f).astype(np.uint32) for f in add_mask_tif_files], axis=0)
#         return raw_vol, seg_vol, add_mask_vol
#     except Exception as e:
#         print (67)
#         return None, None, None

# def create_segmented_cube(
#     raw_vol: np.ndarray,
#     seg_vol: np.ndarray,
#     add_mask_vol: np.ndarray,
#     central_coord: Tuple[int, int, int],
#     side1_coord: Tuple[int, int, int],
#     side2_coord: Tuple[int, int, int],
#     segmentation_type: int,
#     subvolume_size: int = 80,
#     alpha: float = 0.3,
#     input_mask: bool = False  # New parameter
# ) -> np.ndarray:
#     def create_segment_masks(segmentation_volume, s1_coord, s2_coord):
#         x1, y1, z1 = s1_coord
#         x2, y2, z2 = s2_coord
#         seg_id_1 = segmentation_volume[z1, y1, x1]
#         seg_id_2 = segmentation_volume[z2, y2, x2]

#         mask_1 = (segmentation_volume == seg_id_1) if seg_id_1 != 0 else np.zeros_like(segmentation_volume, dtype=bool)
#         mask_2 = (segmentation_volume == seg_id_2) if seg_id_2 != 0 else np.zeros_like(segmentation_volume, dtype=bool)
#         return mask_1, mask_2

#     mask_1_full, mask_2_full = create_segment_masks(seg_vol, side1_coord, side2_coord)
#     mask_3_full = (add_mask_vol > 0)

#     half_size = subvolume_size // 2
#     cx, cy, cz = central_coord
#     x_start, x_end = max(cx - half_size, 0), min(cx + half_size, raw_vol.shape[2])
#     y_start, y_end = max(cy - half_size, 0), min(cy + half_size, raw_vol.shape[1])
#     z_start, z_end = max(cz - half_size, 0), min(cz + half_size, raw_vol.shape[0])

#     sub_raw = raw_vol[z_start:z_end, y_start:y_end, x_start:x_end]
#     sub_mask_1 = mask_1_full[z_start:z_end, y_start:y_end, x_start:x_end]
#     sub_mask_2 = mask_2_full[z_start:z_end, y_start:y_end, x_start:x_end]
#     sub_mask_3 = mask_3_full[z_start:z_end, y_start:y_end, x_start:x_end]

#     pad_z = subvolume_size - sub_raw.shape[0]
#     pad_y = subvolume_size - sub_raw.shape[1]
#     pad_x = subvolume_size - sub_raw.shape[2]

#     if pad_z > 0 or pad_y > 0 or pad_x > 0:
#         sub_raw = np.pad(sub_raw, ((0, pad_z), (0, pad_y), (0, pad_x)), mode='constant', constant_values=0)
#         sub_mask_1 = np.pad(sub_mask_1, ((0, pad_z), (0, pad_y), (0, pad_x)), mode='constant', constant_values=False)
#         sub_mask_2 = np.pad(sub_mask_2, ((0, pad_z), (0, pad_y), (0, pad_x)), mode='constant', constant_values=False)
#         sub_mask_3 = np.pad(sub_mask_3, ((0, pad_z), (0, pad_y), (0, pad_x)), mode='constant', constant_values=False)

#     sub_raw = sub_raw[:subvolume_size, :subvolume_size, :subvolume_size]
#     sub_mask_1 = sub_mask_1[:subvolume_size, :subvolume_size, :subvolume_size]
#     sub_mask_2 = sub_mask_2[:subvolume_size, :subvolume_size, :subvolume_size]
#     sub_mask_3 = sub_mask_3[:subvolume_size, :subvolume_size, :subvolume_size]

#     overlaid_cube = np.zeros((subvolume_size, subvolume_size, 3, subvolume_size), dtype=np.uint8)

#     for z in range(subvolume_size):
#         raw_slice = sub_raw[z].astype(np.float32)
#         mn, mx = raw_slice.min(), raw_slice.max()
#         if mx > mn:
#             raw_slice = (raw_slice - mn) / (mx - mn)
#         else:
#             raw_slice = raw_slice - mn

#         if input_mask:  # New masking logic
#             combined_mask = np.zeros_like(raw_slice, dtype=np.float32)
#             if segmentation_type in [1, 3, 5]:
#                 combined_mask = np.logical_or(combined_mask, sub_mask_1[z])
#             if segmentation_type in [2, 3, 5]:
#                 combined_mask = np.logical_or(combined_mask, sub_mask_2[z])
#             if segmentation_type in [4, 5]:
#                 combined_mask = np.logical_or(combined_mask, sub_mask_3[z])

#             masked_raw = raw_slice * combined_mask
#             masked_rgb = np.stack([masked_raw]*3, axis=-1)
#             overlaid_image = (masked_rgb * 255).astype(np.uint8)
#         else:  # Original overlay logic
#             raw_rgb = np.stack([raw_slice]*3, axis=-1)
#             mask1_rgb = np.zeros_like(raw_rgb)
#             mask2_rgb = np.zeros_like(raw_rgb)
#             mask3_rgb = np.zeros_like(raw_rgb)

#             if segmentation_type in [1, 3, 5]:
#                 mask1_rgb[sub_mask_1[z]] = [1, 0, 0]
#             if segmentation_type in [2, 3, 5]:
#                 mask2_rgb[sub_mask_2[z]] = [0, 0, 1]
#             if segmentation_type in [4, 5]:
#                 mask3_rgb[sub_mask_3[z]] = [0, 1, 0]

#             combined_masks = mask1_rgb + mask2_rgb + mask3_rgb
#             combined_masks = np.clip(combined_masks, 0, 1)
#             overlaid_image = (1 - alpha) * raw_rgb + alpha * combined_masks
#             overlaid_image = (np.clip(overlaid_image, 0, 1) * 255).astype(np.uint8)

#     # Add grayscale conversion option
#     if input_mask:  # If using masks, convert to grayscale
#         overlaid_cube = np.dot(overlaid_cube[..., :3], [0.2989, 0.5870, 0.1140])
#         overlaid_cube = np.expand_dims(overlaid_cube, 2)  # Add channel dimension

#         overlaid_cube[:, :, :, z] = overlaid_image

#     return overlaid_cube

# class VideoMAEDataset(Dataset):
#     def __init__(self, vol_data_dict: dict, synapse_df: pd.DataFrame, processor,
#                  segmentation_type: int, subvol_size: int = 80, num_frames: int = 16,
#                  alpha: float = 0.3, input_mask: bool = False, output_channels: int = 1):
#         self.vol_data_dict = vol_data_dict
#         self.synapse_df = synapse_df.reset_index(drop=True)
#         self.processor = processor
#         self.segmentation_type = segmentation_type
#         self.subvol_size = subvol_size
#         self.num_frames = num_frames
#         self.alpha = alpha
#         self.input_mask = input_mask
#         self.output_channels = output_channels

#     def __len__(self):
#         return len(self.synapse_df)

#     def __getitem__(self, idx):
#         syn_info = self.synapse_df.iloc[idx]
#         bbox_name = syn_info['bbox_name']
#         raw_vol, seg_vol, add_mask_vol = self.vol_data_dict.get(bbox_name, (None, None, None))

#         if raw_vol is None:
#             return torch.zeros((self.num_frames, self.output_channels,
#                               self.subvol_size, self.subvol_size),
#                              dtype=torch.float32), syn_info, bbox_name

#         central_coord = (int(syn_info['central_coord_1']), int(syn_info['central_coord_2']),
#                         int(syn_info['central_coord_3']))
#         side1_coord = (int(syn_info['side_1_coord_1']), int(syn_info['side_1_coord_2']),
#                       int(syn_info['side_1_coord_3']))
#         side2_coord = (int(syn_info['side_2_coord_1']), int(syn_info['side_2_coord_2']),
#                       int(syn_info['side_2_coord_3']))

#         overlaid_cube = create_segmented_cube(
#             raw_vol=raw_vol,
#             seg_vol=seg_vol,
#             add_mask_vol=add_mask_vol,
#             central_coord=central_coord,
#             side1_coord=side1_coord,
#             side2_coord=side2_coord,
#             segmentation_type=self.segmentation_type,
#             subvolume_size=self.subvol_size,
#             alpha=self.alpha,
#             input_mask=self.input_mask
#         )

#         # Convert to grayscale if needed
#         if self.output_channels == 1:
#             overlaid_cube = np.dot(overlaid_cube[..., :3], [0.2989, 0.5870, 0.1140])
#             overlaid_cube = np.expand_dims(overlaid_cube, 2)  # Add channel dimension

#         frames = [overlaid_cube[..., z] for z in range(overlaid_cube.shape[3])]
#         if len(frames) < self.num_frames:
#             frames += [frames[-1]] * (self.num_frames - len(frames))
#         elif len(frames) > self.num_frames:
#             indices = np.linspace(0, len(frames)-1, self.num_frames, dtype=int)
#             frames = [frames[i] for i in indices]

#         inputs = self.processor(frames, return_tensors="pt")
#         return inputs["pixel_values"].squeeze(0).float(), syn_info, bbox_name

# def parse_args():
#     parser = argparse.ArgumentParser(description="VideoMAE Pre-training Script with Segmented Videos and Additional Masks")
#     parser.add_argument('--raw_base_dir', type=str, default='raw')
#     parser.add_argument('--seg_base_dir', type=str, default='seg')
#     parser.add_argument('--add_mask_base_dir', type=str, default='')
#     parser.add_argument('--bbox_name', type=str, default=['bbox1'], nargs='+')
#     parser.add_argument('--excel_file', type=str, default='')
#     parser.add_argument('--csv_output_dir', type=str, default='csv_outputs')
#     parser.add_argument('--checkpoint_dir', type=str, default='checkpoints')
#     parser.add_argument('--log_dir', type=str, default='logs')
#     parser.add_argument('--size', type=tuple, default=(80,80))
#     parser.add_argument('--batch_size', type=int, default=2)
#     parser.add_argument('--num_epochs', type=int, default=5)
#     parser.add_argument('--learning_rate', type=float, default=1e-4)
#     parser.add_argument('--weight_decay', type=float, default=1e-2)
#     parser.add_argument('--subvol_size', type=int, default=80)
#     parser.add_argument('--num_frames', type=int, default=80)
#     parser.add_argument('--mask_ratio', type=float, default=0.75)
#     parser.add_argument('--patience', type=int, default=3)
#     parser.add_argument('--resume_checkpoint', type=str, default=None)
#     parser.add_argument('--save_gifs_dir', type=str, default='gifs')
#     parser.add_argument('--num_gifs', type=int, default=10)
#     parser.add_argument('--alpha', type=float, default=0.3)
#     parser.add_argument('--segmentation_type', type=int, default=5, choices=range(0, 6))
#     parser.add_argument('--input_mask', action='store_true', # Added argument
#                        help='Mask input image using segmentation_type regions')
#     parser.add_argument('--input_channels', type=int, default=1,
#                        help='Number of input channels (1 for grayscale)')
#     parser.add_argument('--grayscale', action='store_true',
#                        help='Convert input to grayscale')
#     parser.add_argument("--extract_features", action="store_true")

#     args, _ = parser.parse_known_args()

#     return args



# class Args:
#     def __init__(self):
#         # Data directories
#         self.raw_base_dir = 'raw'
#         self.seg_base_dir = 'seg'
#         self.add_mask_base_dir = ''
#         self.bbox_name = ['bbox1']
#         self.excel_dir = ''
#         self.checkpoint='hemibrain_production.checkpoint'
#         # Output directories
#         self.csv_output_dir = 'csv_outputs'
#         # self.checkpoint_dir = 'hemibrain_production.checkpoint'
#         self.log_dir = 'logs'
#         self.num_classes=7
#         # Training parameters
#         self.size = (80, 80)
#         self.batch_size = 2
#         self.num_epochs = 5
#         self.learning_rate = 1e-4
#         self.weight_decay = 1e-2
#         self.subvol_size = 80
#         self.num_frames = 80
#         self.mask_ratio = 0.75
#         self.patience = 3
#         self.resume_checkpoint = None

#         # GIF parameters
#         self.save_gifs_dir = 'gifs'
#         self.num_gifs = 10
#         self.alpha = 0.3
#         self.segmentation_type = 5
#         self.input_mask = True  # Set to True if you want masking
#         self.extract_features=True
#         self.num_workers=2
#         self.output_dir='output'
#         self.grayscale=True
#         self.input_channels=1

# # In your notebook cell:
# args = Args()  # Create config object
# args.input_mask = False  # Optionally override parameters
# args.bbox_name = ['bbox1']  # Example override

# # def main(args):
#     # Load data
# vol_data_dict = {}
# for bbox in args.bbox_name:
#     raw, seg, mask = load_volumes(
#         bbox, args.raw_base_dir,
#         args.seg_base_dir, args.add_mask_base_dir
#     )
#     if raw is not None:
#         vol_data_dict[bbox] = (raw, seg, mask)

# # Load annotations
# synapse_dfs = []
# for bbox in args.bbox_name:
#     excel_path = os.path.join(args.excel_dir, f"{bbox}.xlsx")
#     if os.path.exists(excel_path):
#         df = pd.read_excel(excel_path)
#         df["bbox_name"] = bbox
#         synapse_dfs.append(df)
# syn_df = pd.concat(synapse_dfs, ignore_index=True)

# # Create dataset
# processor = SimpleVideoProcessor(size=(args.subvol_size, args.subvol_size))
# dataset = VideoMAEDataset(
#     vol_data_dict=vol_data_dict,
#     synapse_df=syn_df,
#     processor=processor,
#     segmentation_type=args.segmentation_type,
#     subvol_size=args.subvol_size,
#     num_frames=args.num_frames,
#     alpha=args.alpha,
#     input_mask=args.input_mask
# )

GIF saved successfully!


In [ ]:
import logging
import os
import glob
import argparse
import numpy as np
import pandas as pd
import imageio.v3 as iio
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from sklearn.model_selection import train_test_split
import os
import glob
import io
import argparse
import multiprocessing
from typing import List, Tuple

import numpy as np
import pandas as pd
import imageio.v3 as iio
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from torchvision import transforms

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def main(args):
    # Load data
    vol_data_dict = {}
    for bbox in args.bbox_name:
        raw, seg, mask = load_volumes(
            bbox, args.raw_base_dir,
            args.seg_base_dir, args.add_mask_base_dir
        )
        if raw is not None:
            vol_data_dict[bbox] = (raw, seg, mask)

    # Load annotations
    synapse_dfs = []
    for bbox in args.bbox_name:
        excel_path = os.path.join(args.excel_dir, f"{bbox}.xlsx")
        if os.path.exists(excel_path):
            df = pd.read_excel(excel_path)
            df["bbox_name"] = bbox
            synapse_dfs.append(df)
    syn_df = pd.concat(synapse_dfs, ignore_index=True)

    # Create dataset
    processor = SimpleVideoProcessor(size=(args.subvol_size, args.subvol_size))
    dataset = VideoMAEDataset(
        vol_data_dict=vol_data_dict,
        synapse_df=syn_df,
        processor=processor,
        segmentation_type=args.segmentation_type,
        subvol_size=args.subvol_size,
        num_frames=args.num_frames,
        alpha=args.alpha,
        input_mask=args.input_mask
    )

#     # Initialize model
#     model = Vgg3D(
#         input_size=(args.num_frames, args.subvol_size, args.subvol_size),
#         input_fmaps=1,
#         output_classes=args.num_classes
#     )

#     # Load checkpoint
#     if args.checkpoint:
#         state_dict = torch.load(args.checkpoint)["model_state_dict"]
#         model.load_state_dict(state_dict)
#         logger.info(f"Loaded weights from {args.checkpoint}")

#     if args.extract_features:
#         # Get combined dataframe with features
#         combined_df = extract_features(model, dataset, args)

#         # Save results
#         os.makedirs(args.output_dir, exist_ok=True)

#         # Save full dataset with features
#         output_path = os.path.join(args.output_dir, "metadata_with_features.csv")
#         combined_df.to_csv(output_path, index=False)

#         logger.info(f"Saved combined features and metadata to {output_path}")
#         logger.info(f"Dataset shape: {combined_df.shape}")
#         logger.info(f"Feature columns: {[c for c in combined_df.columns if c.startswith('feat_')]}")

# # Paths and directories
# checkpoint_url = "https://dl.dropboxusercontent.com/scl/fo/mfejaomhu43aa6oqs6zsf/AKMAAgT7OrUtruR0AQXZBy0/hemibrain_production.checkpoint.20220225?rlkey=6cmwxdvehy4ylztvsbgkfnrfc&dl=0"
# checkpoint_path = 'hemibrain_production.checkpoint'

# # Download the checkpoint if it doesn't exist
# if not os.path.exists(checkpoint_path):
#     os.system(f"wget -O {checkpoint_path} '{checkpoint_url}'")
#     print("Downloaded VGG3D checkpoint.")
# else:
#     print("VGG3D checkpoint already exists.")


class Args:
    def __init__(self):
        # Data directories
        self.raw_base_dir = 'raw'
        self.seg_base_dir = 'seg'
        self.add_mask_base_dir = ''
        self.bbox_name = ['bbox1']
        self.excel_dir = ''
        self.checkpoint='hemibrain_production.checkpoint'
        # Output directories
        self.csv_output_dir = 'csv_outputs'
        # self.checkpoint_dir = 'hemibrain_production.checkpoint'
        self.log_dir = 'logs'
        self.num_classes=7
        # Training parameters
        self.size = (80, 80)
        self.batch_size = 2
        self.num_epochs = 5
        self.learning_rate = 1e-4
        self.weight_decay = 1e-2
        self.subvol_size = 80
        self.num_frames = 80
        self.mask_ratio = 0.75
        self.patience = 3
        self.resume_checkpoint = None

        # GIF parameters
        self.save_gifs_dir = 'gifs'
        self.num_gifs = 10
        self.alpha = 0.3
        self.segmentation_type = 5
        self.input_mask = True  # Set to True if you want masking
        self.extract_features=True
        self.num_workers=2
        self.output_dir='output'
        self.grayscale=True
        self.input_channels=1
# In your notebook cell:
args = Args()  # Create config object
args.input_mask = True  # Optionally override parameters
args.bbox_name = ['bbox1','bbox2','bbox3','bbox4','bbox5','bbox6']  # Example override

# Run main program
main(args)
# print(f"Processed {len(cubes)} cubes successfully.")

Downloaded VGG3D checkpoint.


KeyboardInterrupt: 

## Show

In [ ]:

import os
import glob
import io
import argparse
import multiprocessing
from typing import List, Tuple
import torch.nn as nn

import numpy as np
import pandas as pd
import imageio.v3 as iio
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from torchvision import transforms

# import wandb  # Uncomment if using Weights & Biases for logging


class SimpleVideoProcessor:
    def __init__(self, size=(80, 80), mean=(0.485, 0.456, 0.406),
                 std=(0.229, 0.224, 0.225)):
        """
        Initializes the processor with resizing and normalization transforms.

        Args:
            size (tuple): Desired output size (height, width).
            mean (tuple): Mean for normalization.
            std (tuple): Standard deviation for normalization.
        """
        self.transform = transforms.Compose([
            transforms.ToPILImage(),          # Convert NumPy array to PIL Image
            transforms.Resize(size),          # Resize to desired size
            transforms.ToTensor(),            # Convert PIL Image to Tensor
            transforms.Normalize(mean=mean, std=std),  # Normalize
        ])

    def __call__(self, frames, return_tensors=None):
        """
        Processes a list of frames.

        Args:
            frames (List[np.ndarray]): List of frames as NumPy arrays.
            return_tensors (str, optional): Type of tensors to return. Defaults to None.

        Returns:
            dict or torch.Tensor: Dictionary containing processed pixel values or tensor.
        """
        # Apply transformations to each frame
        processed_frames = [self.transform(frame) for frame in frames]

        # Stack frames to create a tensor of shape (num_frames, 3, H, W)
        pixel_values = torch.stack(processed_frames)

        if return_tensors == "pt":
            return {"pixel_values": pixel_values}
        else:
            return pixel_values


def load_volumes(bbox_name: str, raw_base_dir: str, seg_base_dir: str, add_mask_base_dir: str) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    """
    Load raw volume, segmentation volume, and additional mask volume for a bounding box.

    Args:
        bbox_name (str): Name of the bounding box directory (e.g., 'bbox1').
        raw_base_dir (str): Base directory for raw data.
        seg_base_dir (str): Base directory for segmentation data.
        add_mask_base_dir (str): Base directory for additional masks.

    Returns:
        tuple: (raw_vol, seg_vol, add_mask_vol) each as np.ndarray
    """
    raw_dir = os.path.join(raw_base_dir, bbox_name)
    seg_dir = os.path.join(seg_base_dir, bbox_name)

    # Transform 'bbox1' to 'bbox_1' for additional masks
    if bbox_name.startswith("bbox"):
        bbox_num = bbox_name.replace("bbox", "")
        add_mask_dir = os.path.join(add_mask_base_dir, f"bbox_{bbox_num}")
    else:
        add_mask_dir = os.path.join(add_mask_base_dir, bbox_name)

    raw_tif_files = sorted(glob.glob(os.path.join(raw_dir, 'slice_*.tif')))
    seg_tif_files = sorted(glob.glob(os.path.join(seg_dir, 'slice_*.tif')))
    add_mask_tif_files = sorted(glob.glob(os.path.join(add_mask_dir, 'slice_*.tif')))

    if len(raw_tif_files) == 0:
        print(f"No raw files found for {bbox_name} in {raw_dir}")
        return None, None, None

    if len(seg_tif_files) == 0:
        print(f"No segmentation files found for {bbox_name} in {seg_dir}")
        return None, None, None

    if len(add_mask_tif_files) == 0:
        print(f"No additional mask files found for {bbox_name} in {add_mask_dir}")
        return None, None, None

    if not (len(raw_tif_files) == len(seg_tif_files) == len(add_mask_tif_files)):
        print(f"Mismatch in number of raw, seg, and additional mask slices for {bbox_name}. Skipping.")
        return None, None, None

    try:
        raw_vol = np.stack([iio.imread(f) for f in raw_tif_files], axis=0)  # shape: (Z, Y, X)
        seg_vol = np.stack([iio.imread(f).astype(np.uint32) for f in seg_tif_files], axis=0)
        add_mask_vol = np.stack([iio.imread(f).astype(np.uint32) for f in add_mask_tif_files], axis=0)
        return raw_vol, seg_vol, add_mask_vol
    except Exception as e:
        print(f"Error loading volumes for {bbox_name}: {e}")
        return None, None, None


def create_segmented_cube(
    raw_vol: np.ndarray,
    seg_vol: np.ndarray,
    add_mask_vol: np.ndarray,
    central_coord: Tuple[int, int, int],
    side1_coord: Tuple[int, int, int],
    side2_coord: Tuple[int, int, int],
    segmentation_type: int,
    subvolume_size: int = 80,
    alpha: float = 0.3
) -> np.ndarray:
    """
    Constructs an 80x80x80 segmented 3D cube around the specified synapse coordinates
    and overlays selected segmentation masks on the raw data with specified transparency for each slice.

    Args:
        raw_vol (np.ndarray): Raw volumetric data.
        seg_vol (np.ndarray): Segmentation volumetric data.
        add_mask_vol (np.ndarray): Additional mask volumetric data.
        central_coord (tuple): Central coordinate (x, y, z).
        side1_coord (tuple): Side 1 coordinate (x, y, z).
        side2_coord (tuple): Side 2 coordinate (x, y, z).
        segmentation_type (int): Type of segmentation overlay (0-5).
        subvolume_size (int, optional): Size of the subvolume. Defaults to 80.
        alpha (float, optional): Transparency factor. Defaults to 0.3.

    Returns:
        np.ndarray: Overlaid cube of shape (height, width, 3, depth).
    """

    def create_segment_masks(segmentation_volume, s1_coord, s2_coord):
        x1, y1, z1 = s1_coord
        x2, y2, z2 = s2_coord
        # Validate within volume
        if not (0 <= z1 < segmentation_volume.shape[0] and
                0 <= y1 < segmentation_volume.shape[1] and
                0 <= x1 < segmentation_volume.shape[2]):
            raise ValueError("Side1 coordinates are out of bounds.")

        if not (0 <= z2 < segmentation_volume.shape[0] and
                0 <= y2 < segmentation_volume.shape[1] and
                0 <= x2 < segmentation_volume.shape[2]):
            raise ValueError("Side2 coordinates are out of bounds.")

        seg_id_1 = segmentation_volume[z1, y1, x1]
        seg_id_2 = segmentation_volume[z2, y2, x2]

        # If seg_id == 0, it means no segment at that voxel
        if seg_id_1 == 0:
            mask_1 = np.zeros_like(segmentation_volume, dtype=bool)
        else:
            mask_1 = (segmentation_volume == seg_id_1)

        if seg_id_2 == 0:
            mask_2 = np.zeros_like(segmentation_volume, dtype=bool)
        else:
            mask_2 = (segmentation_volume == seg_id_2)

        return mask_1, mask_2

    # Build masks
    mask_1_full, mask_2_full = create_segment_masks(seg_vol, side1_coord, side2_coord)
    mask_3_full = (add_mask_vol > 0)  # Assuming binary masks; adjust if necessary

    # Define subvolume bounds
    half_size = subvolume_size // 2
    cx, cy, cz = central_coord

    x_start, x_end = max(cx - half_size, 0), min(cx + half_size, raw_vol.shape[2])
    y_start, y_end = max(cy - half_size, 0), min(cy + half_size, raw_vol.shape[1])
    z_start, z_end = max(cz - half_size, 0), min(cz + half_size, raw_vol.shape[0])

    # Extract subvolumes
    sub_raw = raw_vol[z_start:z_end, y_start:y_end, x_start:x_end]
    sub_mask_1 = mask_1_full[z_start:z_end, y_start:y_end, x_start:x_end]
    sub_mask_2 = mask_2_full[z_start:z_end, y_start:y_end, x_start:x_end]
    sub_mask_3 = mask_3_full[z_start:z_end, y_start:y_end, x_start:x_end]

    # Pad if smaller than subvolume_size
    pad_z = subvolume_size - sub_raw.shape[0]
    pad_y = subvolume_size - sub_raw.shape[1]
    pad_x = subvolume_size - sub_raw.shape[2]

    if pad_z > 0 or pad_y > 0 or pad_x > 0:
        sub_raw = np.pad(sub_raw, ((0, pad_z), (0, pad_y), (0, pad_x)),
                         mode='constant', constant_values=0)
        sub_mask_1 = np.pad(sub_mask_1, ((0, pad_z), (0, pad_y), (0, pad_x)),
                            mode='constant', constant_values=False)
        sub_mask_2 = np.pad(sub_mask_2, ((0, pad_z), (0, pad_y), (0, pad_x)),
                            mode='constant', constant_values=False)
        sub_mask_3 = np.pad(sub_mask_3, ((0, pad_z), (0, pad_y), (0, pad_x)),
                            mode='constant', constant_values=False)

    # Slice to exact shape
    sub_raw = sub_raw[:subvolume_size, :subvolume_size, :subvolume_size]
    sub_mask_1 = sub_mask_1[:subvolume_size, :subvolume_size, :subvolume_size]
    sub_mask_2 = sub_mask_2[:subvolume_size, :subvolume_size, :subvolume_size]
    sub_mask_3 = sub_mask_3[:subvolume_size, :subvolume_size, :subvolume_size]

    # We'll build an overlaid cube: shape => (H, W, 3, D)
    overlaid_cube = np.zeros((subvolume_size, subvolume_size, 3, subvolume_size), dtype=np.uint8)

    # Define colors
    side1_color = np.array([1, 0, 0], dtype=np.float32)           # Red
    side2_color = np.array([0, 0, 1], dtype=np.float32)           # Blue
    vesicles_color = np.array([0, 1, 0], dtype=np.float32)        # Green

    for z in range(subvolume_size):
        # Normalize raw slice to [0, 1]
        raw_slice = sub_raw[z].astype(np.float32)
        mn, mx = raw_slice.min(), raw_slice.max()
        if mx > mn:
            raw_slice = (raw_slice - mn) / (mx - mn)
        else:
            raw_slice = raw_slice - mn  # all zeros if mn=mx

        raw_rgb = np.stack([raw_slice]*3, axis=-1)  # shape (H, W, 3)

        # Initialize colored masks
        mask1_rgb = np.zeros_like(raw_rgb)
        mask2_rgb = np.zeros_like(raw_rgb)
        mask3_rgb = np.zeros_like(raw_rgb)

        # Overlay masks based on segmentation_type
        if segmentation_type in [1, 3, 5]:
            mask1_rgb[sub_mask_1[z]] = side1_color
        if segmentation_type in [2, 3, 5]:
            mask2_rgb[sub_mask_2[z]] = side2_color
        if segmentation_type in [4, 5]:
            mask3_rgb[sub_mask_3[z]] = vesicles_color

        # Combine masks
        combined_masks = mask1_rgb + mask2_rgb + mask3_rgb
        # Ensure that combined masks do not exceed 1
        combined_masks = np.clip(combined_masks, 0, 1)

        # Blend raw image with masks
        overlaid_image = (1 - alpha) * raw_rgb + alpha * combined_masks
        overlaid_image = np.clip(overlaid_image, 0, 1)

        # Convert to uint8
        overlaid_image = (overlaid_image * 255).astype(np.uint8)
        overlaid_cube[:, :, :, z] = overlaid_image

    return overlaid_cube

class VideoMAEDataset(Dataset):
    """
    Dataset class that uses segmented volumes (side1 & side2) and additional masks for VideoMAE pre-training.
    """
    def __init__(self, vol_data_list: List[Tuple[np.ndarray, np.ndarray, np.ndarray]],
                 synapse_df: pd.DataFrame,
                 processor,
                 segmentation_type: int,
                 subvol_size: int = 80,
                 num_frames: int = 16,
                 alpha: float = 0.3):
        """
        Args:
            vol_data_list (List[Tuple[np.ndarray, np.ndarray, np.ndarray]]): List of (raw_vol, seg_vol, add_mask_vol).
            synapse_df (pd.DataFrame): DataFrame with synapse coordinates (central, side1, side2).
            processor: Processor for VideoMAE.
            segmentation_type (int): Type of segmentation overlay (0-5).
            subvol_size (int): Size of the sub-volume to extract.
            num_frames (int): Number of frames for the model.
            alpha (float): Blending alpha for segmentation.
        """
        self.vol_data_list = vol_data_list
        self.synapse_df = synapse_df.reset_index(drop=True)
        self.processor = processor
        self.segmentation_type = segmentation_type
        self.subvol_size = subvol_size
        self.num_frames = num_frames
        self.alpha = alpha

    def __len__(self):
        return len(self.synapse_df)
class VideoMAEDataset(Dataset):
    """
    Dataset class that uses segmented volumes (side1 & side2) and additional masks for VideoMAE pre-training.
    """
    def __init__(self, vol_data_dict: dict,
                 synapse_df: pd.DataFrame,
                 processor,
                 segmentation_type: int,
                 subvol_size: int = 80,
                 num_frames: int = 16,
                 alpha: float = 0.3):
        """
        Args:
            vol_data_dict (dict): Dictionary with keys as bbox_name and values as tuples (raw_vol, seg_vol, add_mask_vol).
            synapse_df (pd.DataFrame): DataFrame with synapse coordinates (central, side1, side2).
            processor: Processor for VideoMAE.
            segmentation_type (int): Type of segmentation overlay (0-5).
            subvol_size (int): Size of the sub-volume to extract.
            num_frames (int): Number of frames for the model.
            alpha (float): Blending alpha for segmentation.
        """
        self.vol_data_dict = vol_data_dict  # Changed to dictionary
        self.synapse_df = synapse_df.reset_index(drop=True)
        self.processor = processor
        self.segmentation_type = segmentation_type
        self.subvol_size = subvol_size
        self.num_frames = num_frames
        self.alpha = alpha

    def __len__(self):
        return len(self.synapse_df)
    def __getitem__(self, idx):
        syn_info = self.synapse_df.iloc[idx]
        bbox_name = syn_info['bbox_name']  # Use bbox_name instead of bbox_index

        # Unpack the volumes using bbox_name instead of bbox_index
        raw_vol, seg_vol, add_mask_vol = self.vol_data_dict.get(bbox_name, (None, None, None))

        if raw_vol is None or seg_vol is None or add_mask_vol is None:
            # Return dummy data if volumes not found
            pixel_values = torch.zeros((self.num_frames, 3, self.subvol_size, self.subvol_size), dtype=torch.float32)
            return pixel_values, syn_info, bbox_name

        # Coordinates
        central_coord = (
            int(syn_info['central_coord_1']),
            int(syn_info['central_coord_2']),
            int(syn_info['central_coord_3'])
        )
        side1_coord = (
            int(syn_info['side_1_coord_1']),
            int(syn_info['side_1_coord_2']),
            int(syn_info['side_1_coord_3'])
        )
        side2_coord = (
            int(syn_info['side_2_coord_1']),
            int(syn_info['side_2_coord_2']),
            int(syn_info['side_2_coord_3'])
        )

        # Create the overlaid segmented cube with the additional mask based on segmentation_type
        overlaid_cube = create_segmented_cube(
            raw_vol=raw_vol,
            seg_vol=seg_vol,
            add_mask_vol=add_mask_vol,
            central_coord=central_coord,
            side1_coord=side1_coord,
            side2_coord=side2_coord,
            segmentation_type=self.segmentation_type,
            subvolume_size=self.subvol_size,
            alpha=self.alpha
        )  # shape: (80, 80, 3, 80)

        # We interpret the last dimension (depth) as frames
        frames = []
        for z in range(overlaid_cube.shape[3]):  # 80 slices
            frame_rgb = overlaid_cube[..., z]  # (80, 80, 3)
            frames.append(frame_rgb)

        # Now reduce or expand to self.num_frames
        total_slices = len(frames)  # 80
        if total_slices < self.num_frames:
            while len(frames) < self.num_frames:
                frames.append(frames[-1])
        elif total_slices > self.num_frames:
            indices = np.linspace(0, total_slices - 1, self.num_frames, dtype=int)
            frames = [frames[i] for i in indices]

        # Process using the VideoMAEImageProcessor
        inputs = self.processor(frames, return_tensors="pt")
        pixel_values = inputs["pixel_values"].squeeze(0)  # (num_frames, 3, H, W)
        pixel_values = pixel_values.float()

        # Return pixel values, the corresponding DataFrame row, and the bbox name
        return pixel_values, syn_info, bbox_name  # Return the pixel values, DataFrame row, and bbox_name

def parse_args():
    """
    Parse command-line arguments for configurable paths and training parameters.
    """
    parser = argparse.ArgumentParser(description="VideoMAE Pre-training Script with Segmented Videos and Additional Masks")

    # Data directories
    parser.add_argument('--raw_base_dir', type=str, default='raw', help='Path to raw data directory')
    parser.add_argument('--seg_base_dir', type=str, default='seg', help='Path to segmentation data directory')
    parser.add_argument('--add_mask_base_dir', type=str, default='', help='Path to additional masks directory')
    parser.add_argument('--bbox_name', type=str, default=['bbox1'], help='Name of the bounding box directory')
    parser.add_argument('--excel_file', type=str, default='', help='Excel file with synapse coordinates')
    # Output and logging directories
    parser.add_argument('--csv_output_dir', type=str, default='csv_outputs', help='Directory to save CSV outputs')
    parser.add_argument('--checkpoint_dir', type=str, default='checkpoints', help='Directory to save model checkpoints')
    parser.add_argument('--log_dir', type=str, default='logs', help='Directory for TensorBoard logs')
    parser.add_argument('--size',type=tuple,default=(80,80),help='Size of the image')
    # Training parameters
    parser.add_argument('--batch_size', type=int, default=2, help='Batch size for training')
    parser.add_argument('--num_epochs', type=int, default=5, help='Number of training epochs')
    parser.add_argument('--learning_rate', type=float, default=1e-4, help='Learning rate for optimizer')
    parser.add_argument('--weight_decay', type=float, default=1e-2, help='Weight decay for optimizer')
    parser.add_argument('--subvol_size', type=int, default=80, help='Size of the sub-volume to extract')
    parser.add_argument('--num_frames', type=int, default=80, help='Number of frames per video clip')
    parser.add_argument('--mask_ratio', type=float, default=0.75, help='Mask ratio for VideoMAE')
    parser.add_argument('--patience', type=int, default=3, help='Patience for early stopping')
    parser.add_argument('--resume_checkpoint', type=str, default=None, help='Path to resume checkpoint')

    # GIF saving parameters
    parser.add_argument('--save_gifs_dir', type=str, default='gifs', help='Directory to save sample GIFs')
    parser.add_argument('--num_gifs', type=int, default=10, help='Number of sample GIFs to save')
    parser.add_argument('--alpha', type=float, default=0.3, help='Transparency factor for segmentation overlay')
    # New argument for segmentation type
    parser.add_argument('--segmentation_type', type=int, default=0, choices=range(0, 6),
                        help='Type of segmentation overlay:\n'
                             '0 = Raw image\n'
                             '1 = Raw + Side1\n'
                             '2 = Raw + Side2\n'
                             '3 = Raw + Side1 + Side2\n'
                             '4 = Raw + Vesicles\n'
                             '5 = Raw + Side1 + Side2 + Vesicles')

    args, _ = parser.parse_known_args()
    return args


class Args:
    def __init__(self):
        # Data directories
        self.raw_base_dir = 'raw'
        self.seg_base_dir = 'seg'
        self.add_mask_base_dir = ''
        self.bbox_name = ['bbox1']
        self.excel_file = ''
        self.checkpoint='hemibrain_production.checkpoint'
        # Output directories
        self.csv_output_dir = 'csv_outputs'
        # self.checkpoint_dir = 'hemibrain_production.checkpoint'
        self.log_dir = 'logs'
        self.num_classes=7
        # Training parameters
        self.size = (80, 80)
        self.batch_size = 2
        self.num_epochs = 5
        self.learning_rate = 1e-4
        self.weight_decay = 1e-2
        self.subvol_size = 80
        self.num_frames = 80
        self.mask_ratio = 0.75
        self.patience = 3
        self.resume_checkpoint = None

        # GIF parameters
        self.save_gifs_dir = 'gifs'
        self.num_gifs = 10
        self.alpha = 0.3
        self.segmentation_type = 0
        self.input_mask = True  # Set to True if you want masking
        self.extract_features=True
        self.num_workers=2
        self.output_dir='output_seg0_grayscale'
        self.grayscale=True
        self.input_channels=1
def main(args):
    # Initialize processor
    args=Args()
    processor = SimpleVideoProcessor(size=(80, 80))

    # List of all bboxes
    # bboxes = [f"bbox{i}" for i in range(1, 8)]  # bbox1 to bbox7
    bboxes =args.bbox_name # bbox1 to bbox7

    # Load volumes for all bboxes
    vol_data_dict = {}
    for bbox_name in args.bbox_name:
        print(bbox_name)
        raw_vol, seg_vol, add_mask_vol = load_volumes(
            bbox_name=bbox_name,
            raw_base_dir=args.raw_base_dir,
            seg_base_dir=args.seg_base_dir,
            add_mask_base_dir=args.add_mask_base_dir
        )
        # print(raw_vol)
        if raw_vol is not None and seg_vol is not None and add_mask_vol is not None:
            vol_data_dict[bbox_name] = (raw_vol, seg_vol, add_mask_vol)
        else:
            print(f"Skipping {bbox_name} due to missing volumes.")

    # Load synapse data for all bboxes
    synapse_dfs = []
    for bbox_name in bboxes:
        excel_file_path = os.path.join(args.excel_file, f"{bbox_name}.xlsx")
        if os.path.exists(excel_file_path):
            df = pd.read_excel(excel_file_path)
            df['bbox_name'] = bbox_name  # Add bbox_name column
            synapse_dfs.append(df)
        else:
            print(f"Excel file not found for {bbox_name}. Skipping.")
    syn_df = pd.concat(synapse_dfs, ignore_index=True)

    # Create dataset
    dataset = VideoMAEDataset(
        vol_data_dict=vol_data_dict,
        synapse_df=syn_df,
        processor=processor,
        segmentation_type=args.segmentation_type,
        subvol_size=args.subvol_size,
        num_frames=args.num_frames,
        alpha=args.alpha
    )

    # Process cubes and collect synapse data
    # cubes = []
    # syn_info_list = []  # List to collect synapse information

    # for idx in range(len(dataset)):
    #     pixel_values, syn_info,bbox_name = dataset[idx]
    #     cubes.append(pixel_values)

    #     # Collect synapse info
    #     syn_info_list.append(syn_info)

    # # Merge all synapse info into a single DataFrame
    # merged_syn_info = pd.DataFrame(syn_info_list)

    # print(f"Processed {len(cubes)} cubes successfully.")
    # # return cubes, merged_syn_info


# if __name__ == "__main__":
#     args = parse_args()
#     main(args)

    # Initialize model
    model = Vgg3D(
        input_size=(args.num_frames, args.subvol_size, args.subvol_size),
        input_fmaps=1,
        output_classes=args.num_classes
    )

    # Load checkpoint
    # if args.checkpoint:
    state_dict = torch.load("/content/hemibrain_production.checkpoint")
    model.load_state_dict(state_dict)
    logger.info(f"Loaded weights from {args.checkpoint}")

    if args.extract_features:
        # Get combined dataframe with features
        combined_df = extract_features(model, dataset, args)

        # Save results
        os.makedirs(args.output_dir, exist_ok=True)

        # Save full dataset with features
        output_path = os.path.join(args.output_dir, "metadata_with_features.csv")
        combined_df.to_csv(output_path, index=False)

        logger.info(f"Saved combined features and metadata to {output_path}")
        logger.info(f"Dataset shape: {combined_df.shape}")
        logger.info(f"Feature columns: {[c for c in combined_df.columns if c.startswith('feat_')]}")

    # Paths and directories
    checkpoint_url = "https://dl.dropboxusercontent.com/scl/fo/mfejaomhu43aa6oqs6zsf/AKMAAgT7OrUtruR0AQXZBy0/hemibrain_production.checkpoint.20220225?rlkey=6cmwxdvehy4ylztvsbgkfnrfc&dl=0"
    checkpoint_path = 'hemibrain_production.checkpoint'

    # Download the checkpoint if it doesn't exist
    if not os.path.exists(checkpoint_path):
        os.system(f"wget -O {checkpoint_path} '{checkpoint_url}'")
        print("Downloaded VGG3D checkpoint.")
    else:
        print("VGG3D checkpoint already exists.")


if __name__ == "__main__":
    args = parse_args()
    main(args)


# args = parse_args()
# cubes , sys_inf= main(args)
# print(f"Processed {len(cubes)} cubes successfully.")

# bbox1
# Processed 58 cubes successfully.
# Processed 58 cubes successfully.

# import torch
# import numpy as np
# import imageio

# # After running main() and getting the cubes list
# cube = cubes[0]  # Select the first cube (adjust index as needed)

# # Define normalization parameters (must match what's used in SimpleVideoProcessor)
# mean = torch.tensor([0.485, 0.456, 0.406]).view(3, 1, 1)
# std = torch.tensor([0.229, 0.224, 0.225]).view(3, 1, 1)

# # Denormalize the tensor
# denormalized_cube = cube * std + mean

# # Clamp values to valid [0,1] range and convert to numpy
# denormalized_cube = torch.clamp(denormalized_cube, 0, 1)
# frames = denormalized_cube.permute(0, 2, 3, 1).numpy()  # Change to (T, H, W, C)
# frames = (frames * 255).astype(np.uint8)  # Convert to 0-255

# imageio.mimsave('synapse_cube.gif', frames, fps=10)

# print("GIF saved successfully!")

# GIF saved successfully!

bbox1


<ipython-input-4-64b3fea4619f>:669: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load("/content/hemibrain_production.checkpoint")


RuntimeError: PytorchStreamReader failed reading zip archive: failed finding central directory

# SEg Mode =0 (raw)

In [ ]:

class Args:
    def __init__(self):
        # Data directories
        self.raw_base_dir = 'raw'
        self.seg_base_dir = 'seg'
        self.add_mask_base_dir = ''
        self.bbox_name = ['bbox1']
        self.excel_dir = ''
        self.checkpoint='hemibrain_production.checkpoint'
        # Output directories
        self.csv_output_dir = 'csv_outputs'
        # self.checkpoint_dir = 'hemibrain_production.checkpoint'
        self.log_dir = 'logs'
        self.num_classes=7
        # Training parameters
        self.size = (80, 80)
        self.batch_size = 2
        self.num_epochs = 5
        self.learning_rate = 1e-4
        self.weight_decay = 1e-2
        self.subvol_size = 80
        self.num_frames = 80
        self.mask_ratio = 0.75
        self.patience = 3
        self.resume_checkpoint = None

        # GIF parameters
        self.save_gifs_dir = 'gifs'
        self.num_gifs = 10
        self.alpha = 0.3
        self.segmentation_type = 0
        self.input_mask = True  # Set to True if you want masking
        self.extract_features=True
        self.num_workers=2
        self.output_dir='output_seg0_grayscale'
        self.grayscale=True
        self.input_channels=1
# In your notebook cell:
args = Args()  # Create config object
args.input_mask = True  # Optionally override parameters
args.bbox_name = ['bbox1','bbox2','bbox3','bbox4','bbox5','bbox6']  # Example override

# Run main program
main(args)
# print(f"Processed {len(cubes)} cubes successfully.")

<ipython-input-7-8ebce88baa15>:194: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(args.checkpoint)["model_state_dict"]


In [ ]:

class Args:
    def __init__(self):
        # Data directories
        self.raw_base_dir = 'raw'
        self.seg_base_dir = 'seg'
        self.add_mask_base_dir = ''
        self.bbox_name = ['bbox1']
        self.excel_dir = ''
        self.checkpoint='hemibrain_production.checkpoint'
        # Output directories
        self.csv_output_dir = 'csv_outputs'
        # self.checkpoint_dir = 'hemibrain_production.checkpoint'
        self.log_dir = 'logs'
        self.num_classes=7
        # Training parameters
        self.size = (80, 80)
        self.batch_size = 2
        self.num_epochs = 5
        self.learning_rate = 1e-4
        self.weight_decay = 1e-2
        self.subvol_size = 80
        self.num_frames = 80
        self.mask_ratio = 0.75
        self.patience = 3
        self.resume_checkpoint = None

        # GIF parameters
        self.save_gifs_dir = 'gifs'
        self.num_gifs = 10
        self.alpha = 0.3
        self.segmentation_type = 0
        self.input_mask = True  # Set to True if you want masking
        self.extract_features=True
        self.num_workers=2
        self.output_dir='output_seg0_grayscale'
        self.grayscale=True
        self.input_channels=1
# In your notebook cell:
args = Args()  # Create config object
args.input_mask = True  # Optionally override parameters
args.bbox_name = ['bbox1','bbox2','bbox3','bbox4','bbox5','bbox6']  # Example override

# Run main program
main(args)
# print(f"Processed {len(cubes)} cubes successfully.")

# Dim Red and Visualize

In [ ]:
!dir

bbox1.xlsx  bbox6.xlsx			     __MACOSX	      vit_umap_x_vs_y.html
bbox2.xlsx  bbox7.xlsx			     raw	      vit_umap_x_vs_z.html
bbox3.xlsx  csv_outputs			     sample_data      vit_umap_y_vs_z.html
bbox4.xlsx  downloaded_file.zip		     seg
bbox5.xlsx  hemibrain_production.checkpoint  vit_umap3d.html


In [ ]:
import pandas as pd
import numpy as np

from sklearn.decomposition import PCA
import umap.umap_ as umap
import plotly.express as px

merged_csv = '/content/output/metadata_with_features.csv'
df = pd.read_csv(merged_csv)

feat_cols = [c for c in df.columns if c.startswith('feat_')]
X = df[feat_cols].values  # shape: [N, hidden_size] (e.g. [N, 768])

pca = PCA(n_components=7, random_state=42)
X_pca = pca.fit_transform(X)  # shape => [N, 50]

# 4) UMAP from 50 -> 3 dims (for 3D visualization)
umap_3d = umap.UMAP(
    n_components=3,
    n_neighbors=15,     # can tune
    min_dist=0.1,       # can tune
    random_state=42
)
X_umap3 = umap_3d.fit_transform(X_pca)  # shape => [N, 3]

# 5) Add UMAP coordinates back to the DataFrame
df['umap_x'] = X_umap3[:,0]
df['umap_y'] = X_umap3[:,1]
df['umap_z'] = X_umap3[:,2]

fig = px.scatter_3d(
    df,
    x='umap_x',
    y='umap_y',
    z='umap_z',
    color='bbox_name',
    hover_data=['central_coord_1', 'central_coord_2', 'central_coord_3']
)
fig.update_traces(marker=dict(size=3))
fig.update_layout(width=800, height=600)
fig.write_html("VGG_umap3d.html")

fig.show()


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.

/usr/local/lib/python3.11/dist-packages/umap/umap_.py:1952: UserWarning:

n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.



In [ ]:
import plotly.express as px

# 1) UMAP (x vs. y)
fig_xy = px.scatter(
    df,
    x="umap_x",
    y="umap_y",
    color="bbox_name",  # color by bbox_name => discrete legend
    title="UMAP (x vs y)",
    hover_data=["umap_x", "umap_y", "bbox_name", "Var1"]
)
fig_xy.write_html("VGG_umap_x_vs_y.html")

fig_xy.show()

# 2) UMAP (x vs. z)
fig_xz = px.scatter(
    df,
    x="umap_x",
    y="umap_z",
    color="bbox_name",
    title="UMAP (x vs z)",
    hover_data=["umap_x", "umap_z", "bbox_name", "Var1"]
)
fig_xz.write_html("VGG_umap_x_vs_z.html")

fig_xz.show()

# 3) UMAP (y vs. z)
fig_yz = px.scatter(
    df,
    x="umap_y",
    y="umap_z",
    color="bbox_name",
    title="UMAP (y vs z)",
    hover_data=["umap_y", "umap_z", "bbox_name", "Var1"]
)
fig_xz.write_html("VGG_umap_y_vs_z.html")

fig_yz.show()


In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(
    rows=1, cols=3,
    subplot_titles=[
        "UMAP (x vs y)",
        "UMAP (x vs z)",
        "UMAP (y vs z)"
    ]
)


cat_codes = df["bbox_name"].astype("category").cat.codes
trace_xy = go.Scatter(
    x=df["umap_x"],
    y=df["umap_y"],
    mode="markers",
    name="(x vs y)",
    marker=dict(
        color=cat_codes,
        colorscale="Viridis",
        showscale=True,
        size=5
    ),
    text=df["bbox_name"],    # hover text
    hovertemplate="<b>bbox_name:%{text}</b><br>umap_x=%{x}<br>umap_y=%{y}<extra></extra>"
)
fig.add_trace(trace_xy, row=1, col=1)

trace_xz = go.Scatter(
    x=df["umap_x"],
    y=df["umap_z"],
    mode="markers",
    name="(x vs z)",
    marker=dict(
        color=cat_codes,
        colorscale="Viridis",
        showscale=False,  # we already have a colorbar in the first subplot
        size=5
    ),
    text=df["bbox_name"],
    hovertemplate="<b>bbox_name:%{text}</b><br>umap_x=%{x}<br>umap_z=%{y}<extra></extra>"
)
fig.add_trace(trace_xz, row=1, col=2)

# 3) UMAP_y vs UMAP_z
trace_yz = go.Scatter(
    x=df["umap_y"],
    y=df["umap_z"],
    mode="markers",
    name="(y vs z)",
    marker=dict(
        color=cat_codes,
        colorscale="Viridis",
        showscale=False,
        size=5
    ),
    text=df["bbox_name"],
    hovertemplate="<b>bbox_name:%{text}</b><br>umap_y=%{x}<br>umap_z=%{y}<extra></extra>"
)
fig.add_trace(trace_yz, row=1, col=3)

# Adjust layout
fig.update_layout(
    title="2D UMAP Projections (All Pairwise Components)",
    width=1800,   # wide figure
    height=600,
    showlegend=False
)

fig.show()
